In [1]:
#!pip install seaborn
!sudo pip install keras --upgrade

  Using cached Keras-2.0.8-py2.py3-none-any.whl
  Using cached six-1.11.0-py2.py3-none-any.whl
Requirement already up-to-date: pyyaml in /usr/lib64/python2.7/dist-packages (from keras)
Requirement already up-to-date: numpy>=1.9.1 in /usr/lib64/python2.7/dist-packages (from keras)
Requirement already up-to-date: scipy>=0.14 in /usr/lib64/python2.7/dist-packages (from keras)
  Found existing installation: six 1.10.0
    Uninstalling six-1.10.0:
      Successfully uninstalled six-1.10.0
  Found existing installation: Keras 1.2.2
    Uninstalling Keras-1.2.2:
      Successfully uninstalled Keras-1.2.2


In [1]:
from Constants import ROOT_DIR, \
    H5_DIR_FULLYCONNECTED_MULTIPLE_RELU,\
    H5_DIR_FULLYCONNECTED_MULTIPLE_ELU,\
    H5_DIR_COVNET_MULTIPLE_RELU,\
    H5_DIR_COVNET_MULTIPLE_ELU,\
    H5_DIR_COVNET_JUMP_SIGMA,\
    H5_DIR_COVNET_JUMP_MU,\
    H5_DIR_COVNET_LAMBDA,\
    H5_DIR_ENSEMBLING,\
    HISTORY_DIR_FEEDFORWARD_RELU,\
    PREDICTIONS_DIR_FEEDFORWARD_RELU

In [2]:
import numpy as np
import math
import random
#import statsmodels.api as sm
import matplotlib.pyplot as plt
import pylab

from scipy import stats
from numpy import random as nrand

%matplotlib inline

# Optional:
import seaborn as sns
sns.set(color_codes=True)
sns.set_style(style='white')
sns.set(font_scale=1.6)

ImportError: No module named seaborn

In [3]:
import tensorflow as tf
import keras
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Flatten, LSTM, Convolution2D, MaxPooling2D, Input
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [4]:
print(keras.__version__)

2.0.8


In [5]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [6]:
from Merton_Jump_Diffusion_SDE_Utilities import \
    ModelParameters, \
    random_model_params, \
    geometric_brownian_motion_jump_diffusion_log_returns
    
from Models import \
    fullyconnected_multiple_ELUs, \
    covnet_multiple_ELUs_8_layers, \
    covnet_single_ReLUs_6_layers, \
    fullyconnected_multiple_ReLUs, \
    covnet_multiple_ELUs_10_layers, \
    covnet_multiple_ELUs_10_layers_dilated

In [7]:
def test_set_model_params():
    return ModelParameters(
        all_time=2000,
        all_delta=0.00396825396,
        all_sigma = 0.1,
        gbm_mu = 0.05,
        jumps_sigma=0.07,
        jumps_mu=0.05,
        jumps_lamda=0.02)

def standard_convolutional_test_set(sample_size = 1000, dim = (40, 50), mp = None):
    if mp == None:
        mp = test_set_model_params()
        
    test_set = []
    for i in range(sample_size):
        if (i%100 == 0):
            print(i)
        test_set.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))

    test_set_pred = np.reshape(np.array(test_set), (sample_size, dim[0], dim[1], 1))
    return test_set_pred

def convolutional_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5, constraint_all_sigma = (0.001,0.2), constraint_gbm_mu = (-1,1), constraint_jumps_lamda = (0.0001,0.025), constraint_jumps_sigma = (0.001, 0.2), constraint_jumps_mu = (-0.5,0.5)):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params(constraint_all_sigma, constraint_gbm_mu, constraint_jumps_lamda, constraint_jumps_sigma, constraint_jumps_mu)
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append(mp)
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_data = np.reshape(np.array(RETURNS_), (paramset_size*paths_p_paramset, 40, 50, 1))
        sigmas = np.array([[mp.all_sigma] for mp in PARAMS_])
        mus = np.array([[mp.gbm_mu] for mp in PARAMS_])
        jump_sigmas = np.array([[mp.jumps_sigma] for mp in PARAMS_])
        jump_mus = np.array([[mp.jumps_mu] for mp in PARAMS_])
        lambdas = np.array([[mp.lamda] for mp in PARAMS_])
        
        yield train_data, [sigmas, mus, jump_sigmas, jump_mus, lambdas]

def standard_fullyconnected_test_set(sample_size = 1000):
    test_set = []
    for i in range(sample_size):
        if (i%100 == 0):
            print(i)
        test_set.append(geometric_brownian_motion_jump_diffusion_log_returns(test_set_model_params()))

    train_moments = [np.append(stats.moment(example, moment = range(2,20)), np.mean(example)) for example in test_set]
    train_statistics = np.array([np.append(train_moments[i], 
                                  sm.tsa.stattools.acf(test_set[i])) 
                        for i in range(len(test_set))])
    return train_statistics


def standard_test_sets(sample_size = 1000, dim = (40, 50)):
    test_set = []
    for i in range(sample_size):
        if (i%100 == 0):
            print(i)
        test_set.append(geometric_brownian_motion_jump_diffusion_log_returns(test_set_model_params()))

    test_set_pred = np.reshape(np.array(test_set), (sample_size, dim[0], dim[1], 1))
    
    train_moments = [np.append(stats.moment(example, moment = range(2,20)), np.mean(example)) for example in test_set]
    train_statistics = np.array([np.append(train_moments[i], 
                                  sm.tsa.stattools.acf(test_set[i])) 
                        for i in range(len(test_set))])
    return test_set_pred, train_statistics

def fullyconnected_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params()
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append(mp)
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_moments = [np.append(stats.moment(example, moment = range(2,20)), np.mean(example)) for example in RETURNS_]

        train_statistics = np.array([np.append(train_moments[i], 
                              sm.tsa.stattools.acf(RETURNS_[i])) 
                    for i in range(len(RETURNS_))])
        
        sigmas = np.array([[mp.all_sigma] for mp in PARAMS_])
        mus = np.array([[mp.gbm_mu] for mp in PARAMS_])
        jump_sigmas = np.array([[mp.jumps_sigma] for mp in PARAMS_])
        jump_mus = np.array([[mp.jumps_mu] for mp in PARAMS_])
        lambdas = np.array([[mp.lamda] for mp in PARAMS_])
        
        yield train_statistics, [sigmas, mus, jump_sigmas, jump_mus, lambdas]

### Fully Connected Multiple ReLUs

In [ ]:
feedforward = fullyconnected_multiple_ReLUs()
#feedforward.load_weights(H5_DIR_FULLYCONNECTED_MULTIPLE_RELU)

In [ ]:
test_set = standard_fullyconnected_test_set()

In [ ]:
#feedforward_relu_histories = np.load(HISTORY_DIR_FEEDFORWARD_RELU).tolist()
#feedforward_relu_predictions = np.load(PREDICTIONS_DIR_FEEDFORWARD_RELU).tolist()
feedforward_relu_histories = []
feedforward_relu_predictions = []

In [ ]:
train_gen = fullyconnected_batch_generator(paramset_size=1500, paths_p_paramset=1)

In [ ]:
!git pull

In [ ]:
for i in range(1000):
    history = feedforward.fit_generator(train_gen, steps_per_epoch = 10, epochs = 1)
    feedforward_relu_histories.append(history.history)
    feedforward_relu_predictions.append(feedforward.predict(test_set))
    
    feedforward.save_weights(H5_DIR_FULLYCONNECTED_MULTIPLE_RELU)

    np.save(HISTORY_DIR_FEEDFORWARD_RELU, feedforward_relu_histories)
    np.save(PREDICTIONS_DIR_FEEDFORWARD_RELU, feedforward_relu_predictions)

    if i%5 == 0:
        !git config --global user.email "rwmmostert@gmail.com"
        !git config --global user.name "RWMostert"
        !git add --all
        !git commit -am "updated history, predictions and model" 
        !git push -u origin master --quiet -f

# Fully Connected Multiple ELUs

In [ ]:
feedforward = fullyconnected_multiple_ELUs()
feedforward.load_weights(H5_DIR_FULLYCONNECTED_MULTIPLE_ELU)

In [ ]:
feedforward.load_weights(H5_DIR_FULLYCONNECTED_MULTIPLE_ELU)

In [ ]:
train_gen = fullyconnected_batch_generator(paramset_size=50, paths_p_paramset=1)
#test_set = standard_fullyconnected_test_set()

In [ ]:
histories2 = np.load("feedforward_mo_histories.npy").tolist()
ff_predicts = np.load("ff_predicts.npy").tolist()
#histories2 = []
#ff_predicts = []

In [ ]:
for i in range(1000):
    history = feedforward.fit_generator(train_gen, steps_per_epoch = 1000, epochs = 1)
    histories2.append(history.history)
    ff_predicts.append(feedforward.predict(test_set))
    
    feedforward.save_weights(H5_DIR_FULLYCONNECTED_MULTIPLE_ELU)
    np.save("feedforward_mo_histories.npy", histories2)
    np.save("ff_predicts.npy", ff_predicts)
    
    !git config --global user.email "rwmmostert@gmail.com"
    !git config --global user.name "RWMostert"
    !git add --all
    !git commit -am "updated history, predictions and model"
    !git push -u origin master --quiet

# Covnet Multiple Output with ELUs

In [ ]:
covnet_mo_elu = load_model("FULLMODEL_RELAXED_ELU_COVNET.h5")

In [ ]:
#covnet_mo_elu = covnet_multiple_ELUs_8_layers(r_squared = False)
#covnet_mo_elu.load_weights("RELAXED_ELU_COVNET.h5")

In [ ]:
#covnet_mo_elu.save(H5_DIR_COVNET_MULTIPLE_ELU)

In [ ]:
test_set_pred = standard_convolutional_test_set()

In [ ]:
train_gen = convolutional_batch_generator(paramset_size=50, paths_p_paramset=1, constraint_gbm_mu=(-0.5, 0.5))

In [ ]:
histories_elu = np.load("covnet_histories_elu_RELAXED.npy").tolist()
covnet_mo_elu_predictions = np.load("covnet_mo_elu_predictions_RELAXED.npy").tolist()

In [ ]:
for i in range(150000):
    history = covnet_mo_elu.fit_generator(train_gen, samples_per_epoch = 100, nb_epoch = 10)
    
    histories_elu.append(history.history)
    
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    covnet_mo_elu_predictions.append(predictions_elu)
    
    covnet_mo_elu.save("FULLMODEL_RELAXED_ELU_COVNET.h5")
    
    np.save("covnet_histories_elu_RELAXED.npy", histories_elu)
    np.save("covnet_mo_elu_predictions_RELAXED.npy", covnet_mo_elu_predictions)
    
    !git config --global user.email "rwmmostert@gmail.com"
    !git config --global user.name "RWMostert"
    !git add --all
    !git commit -am "updated history, predictions and model"
    !git push -u origin master --quiet

In [ ]:
print()

# DILATED Covnet Multiple Output with ELUs 10 Layers

In [8]:
#covnet_mo_elu = load_model("FULLMODEL_10LAYERS_COVNET.h5", custom_objects={"r2":r2})
covnet_mo_elu = covnet_multiple_ELUs_10_layers_dilated()

/home/carnd/Stochastic-Process-Calibration-Dissertation/Source_Code/Models.py:192: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  merged = merge([layer1, dilated_layer], mode='concat',concat_axis=1) #or 'concat' or 'mul'
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/carnd/Stochastic-Process-Calibration-Dissertation/Source_Code/Models.py:227: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`
  convnet_mo_elu = Model(input = input_1, output=[output1, output2, output3, output4, output5])


In [9]:
#covnet_mo_elu = covnet_multiple_ELUs_10_layers()
#covnet_mo_elu.load_weights("RELAXED_ELU_COVNET.h5")

In [10]:
#covnet_mo_elu.save(H5_DIR_COVNET_MULTIPLE_ELU)

In [11]:
test_set_pred = standard_convolutional_test_set()

0
100
200
300
400
500
600
700
800
900


In [25]:
train_gen = convolutional_batch_generator(paramset_size=1250, paths_p_paramset=1, constraint_gbm_mu=(-0.5, 0.5))

In [13]:
histories_elu = []#np.load("covnet_histories_elu_10LAYERS.npy").tolist()
covnet_mo_elu_predictions = []#np.load("covnet_mo_elu_predictions_10LAYERS.npy").tolist()

In [14]:
smallest_loss = 20

# Covnet Multiple Output with ELUs 10 Layers

In [10]:
covnet_mo_elu = load_model("FULLMODEL_10LAYERS_COVNET.h5", custom_objects={"r2":r2})

In [ ]:
#covnet_mo_elu = covnet_multiple_ELUs_10_layers()
#covnet_mo_elu.load_weights("RELAXED_ELU_COVNET.h5")

In [ ]:
#covnet_mo_elu.save(H5_DIR_COVNET_MULTIPLE_ELU)

In [11]:
test_set_pred = standard_convolutional_test_set()

0
100
200
300
400
500
600
700
800
900


In [16]:
train_gen = convolutional_batch_generator(paramset_size=1750, paths_p_paramset=1, constraint_gbm_mu=(-0.5, 0.5))

In [13]:
histories_elu = np.load("covnet_histories_elu_10LAYERS.npy").tolist()
covnet_mo_elu_predictions = np.load("covnet_mo_elu_predictions_10LAYERS.npy").tolist()

In [14]:
smallest_loss = 20

In [19]:
for i in range(150000):
    history = covnet_mo_elu.fit_generator(train_gen, samples_per_epoch = 10, nb_epoch = 1)
    
    if history.history['loss'][-1]<smallest_loss:
        print("SAVE")
        smallest_loss = history.history['loss'][-1]
        covnet_mo_elu.save("FULLMODEL_10LAYERS_COVNET.h5")
        histories_elu.append(history.history)

        predictions_elu = covnet_mo_elu.predict(test_set_pred)

        covnet_mo_elu_predictions.append(predictions_elu)
    
    if i%5 == 0:
        np.save("covnet_histories_elu_10LAYERS.npy", histories_elu)
        np.save("covnet_mo_elu_predictions_10LAYERS.npy", covnet_mo_elu_predictions)

        !git config --global user.email "rwmmostert@gmail.com"
        !git config --global user.name "RWMostert"
        !git add --all
        !git commit -am "updated history, predictions and model"
        !git push -u origin master --quiet

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=1, steps_per_epoch=10)`
  


Epoch 1/1
10/10 [==============================] - 172s - loss: 0.0069 - sigma_loss: 5.1815e-05 - mu_loss: 0.0024 - jump_sigma_loss: 0.0012 - jump_mu_loss: 0.0033 - lambda_loss: 1.2211e-05 - sigma_mean_absolute_percentage_error: 17.4387 - sigma_r2: 0.9842 - mu_mean_absolute_percentage_error: 82.4425 - mu_r2: 0.9717 - jump_sigma_mean_absolute_percentage_error: 91.5152 - jump_sigma_r2: 0.6515 - jump_mu_mean_absolute_percentage_error: 51.5976 - jump_mu_r2: 0.9599 - lambda_mean_absolute_percentage_error: 50.5395 - lambda_r2: 0.7641    
[master a478fe8] updated history, predictions and model
 1 file changed, 1 insertion(+), 1 deletion(-)
Branch master set up to track remote branch master from origin.
Epoch 1/1
10/10 [==============================] - 172s - loss: 0.0066 - sigma_loss: 5.1216e-05 - mu_loss: 0.0023 - jump_sigma_loss: 0.0011 - jump_mu_loss: 0.0031 - lambda_loss: 1.0300e-05 - sigma_mean_absolute_percentage_error: 16.1368 - sigma_r2: 0.9843 - mu_mean_absolute_percentage_error: 62

KeyboardInterrupt: 

# SAME Covnet Multiple Output with ELUs 10 Layers

In [14]:
import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, LSTM, MaxPooling2D, Input, merge
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras import backend as K

In [15]:
def r2(y_true, y_pred):
    """
    returns the correlation coefficient of y_pred against y_true.

    :param y_true: the true values (independent variable)
    :param y_pred: the predicted values (dependent variable)
    """
    
    SSE = K.sum(K.square(y_true-y_pred))
    SST = K.sum(K.square(y_true-K.mean(y_true)))
    
    return 1-SSE/SST

In [22]:
convnet_mo_elu = load_model("FULLMODEL_6_hidden_LAYERS.h5", custom_objects={"r2":r2})

IOError: Unable to open file (unable to open file: name = 'FULLMODEL_6_hidden_LAYERS.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [23]:
#covnet_mo_elu = load_model("FULLMODEL_10LAYERS_COVNET.h5", custom_objects={"r2":r2})

input_1 = Input(shape = (40, 50, 1))

layer1 = Conv2D(32, (3, 3), activation='elu')(input_1)
layer2 = Conv2D(32, (3, 3), activation='elu')(layer1)
layer3 = MaxPooling2D(pool_size=(2,2))(layer2)

layer4 = Conv2D(64, (3, 3), activation='elu')(layer3)
layer5 = Conv2D(64, (3, 3), activation='elu')(layer4)
layer6 = MaxPooling2D(pool_size=(2,2))(layer5)

flatten = Flatten()(layer6)

output1_layer1 = Dense(32, activation='elu')(flatten)
output1_layer2 = Dense(16, activation='elu')(output1_layer1)
output1 = Dense(1, name="sigma")(output1_layer2)

output2_layer1 = Dense(32, activation='elu')(flatten)
output2_layer2 = Dense(16, activation='elu')(output2_layer1)
output2 = Dense(1, name="mu")(output2_layer2)

output3_layer1 = Dense(32, activation='elu')(flatten)
output3_layer2 = Dense(16, activation='elu')(output3_layer1)
output3 = Dense(1, name="jump_sigma")(output3_layer2)

output4_layer1 = Dense(32, activation='elu')(flatten)
output4_layer2 = Dense(16, activation='elu')(output4_layer1)
output4 = Dense(1, name="jump_mu")(output4_layer2)

output5_layer1 = Dense(32, activation='elu')(flatten)
output5_layer2 = Dense(16, activation='elu')(output5_layer1)
output5 = Dense(1, name="lambda")(output5_layer2)

convnet_mo_elu = Model(input = input_1, output=[output1, output2, output3, output4, output5])
convnet_mo_elu.compile('adam', 'mean_squared_error', metrics=['mean_absolute_percentage_error', r2])

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [20]:
#covnet_mo_elu = covnet_multiple_ELUs_10_layers()
#covnet_mo_elu.load_weights("RELAXED_ELU_COVNET.h5")

In [21]:
#covnet_mo_elu.save(H5_DIR_COVNET_MULTIPLE_ELU)

In [17]:
test_set_pred = standard_convolutional_test_set()

0
100
200
300
400
500
600
700
800
900


In [35]:
train_gen = convolutional_batch_generator(paramset_size=300, paths_p_paramset=5, constraint_gbm_mu=(-0.5, 0.5))

In [25]:
histories_elu = []#np.load("SIMPLE_histories_elu_6_hidden_LAYERS.npy").tolist()
covnet_mo_elu_predictions = []# np.load("SIMPLE_mo_elu_predictions_6_hidden_LAYERS.npy").tolist()

In [24]:
smallest_loss = 20

In [36]:
for i in range(150000):
    history = convnet_mo_elu.fit_generator(train_gen, samples_per_epoch = 10, nb_epoch = 1)
    histories_elu.append(history.history)

    predictions_elu = convnet_mo_elu.predict(test_set_pred)

    covnet_mo_elu_predictions.append(predictions_elu)
    
    if history.history['loss'][-1]<smallest_loss:
        print("SAVE")
        smallest_loss = history.history['loss'][-1]
        convnet_mo_elu.save("SIMPLE_6_hidden_LAYERS.h5")
    
    if i%5 == 0:
        np.save("SIMPLE_histories_elu_6_hidden_LAYERS.npy", histories_elu)
        np.save("SIMPLE_mo_elu_predictions_6_hidden_LAYERS.npy", covnet_mo_elu_predictions)

        !git config --global user.email "rwmmostert@gmail.com"
        !git config --global user.name "RWMostert"
        !git add --all
        !git commit -am "updated history, predictions and model"
        !git push -u origin master --quiet

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=1, steps_per_epoch=10)`
  


Epoch 1/1
10/10 [==============================] - 115s - loss: 0.0136 - sigma_loss: 1.3725e-04 - mu_loss: 0.0034 - jump_sigma_loss: 0.0028 - jump_mu_loss: 0.0072 - lambda_loss: 2.5272e-05 - sigma_mean_absolute_percentage_error: 21.3842 - sigma_r2: 0.9582 - mu_mean_absolute_percentage_error: 76.8772 - mu_r2: 0.9583 - jump_sigma_mean_absolute_percentage_error: 153.4793 - jump_sigma_r2: 0.1153 - jump_mu_mean_absolute_percentage_error: 110.3437 - jump_mu_r2: 0.9134 - lambda_mean_absolute_percentage_error: 109.6344 - lambda_r2: 0.5234    
[master 0ab34b8] updated history, predictions and model
 3 files changed, 3 insertions(+), 3 deletions(-)
^C
Epoch 1/1
10/10 [==============================] - 117s - loss: 0.0134 - sigma_loss: 1.2004e-04 - mu_loss: 0.0035 - jump_sigma_loss: 0.0029 - jump_mu_loss: 0.0069 - lambda_loss: 2.4766e-05 - sigma_mean_absolute_percentage_error: 19.4939 - sigma_r2: 0.9625 - mu_mean_absolute_percentage_error: 137.7743 - mu_r2: 0.9571 - jump_sigma_mean_absolute_perce

KeyboardInterrupt: 

In [26]:
for i in range(150000):
    history = covnet_mo_elu.fit_generator(train_gen, samples_per_epoch = 10, nb_epoch = 1)
    
    if history.history['loss'][-1]<smallest_loss:
        print("SAVE")
        smallest_loss = history.history['loss'][-1]
        covnet_mo_elu.save("FULLMODEL_10LAYERS_COVNET.h5")
        histories_elu.append(history.history)

        predictions_elu = covnet_mo_elu.predict(test_set_pred)

        covnet_mo_elu_predictions.append(predictions_elu)
    
    if i%5 == 0:
        np.save("covnet_histories_elu_10LAYERS.npy", histories_elu)
        np.save("covnet_mo_elu_predictions_10LAYERS.npy", covnet_mo_elu_predictions)

        !git config --global user.email "rwmmostert@gmail.com"
        !git config --global user.name "RWMostert"
        !git add --all
        !git commit -am "updated history, predictions and model"
        !git push -u origin master --quiet

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
  
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=10, epochs=1)`
  


Epoch 1/1
10/10 [==============================] - 124s - loss: 0.0321 - sigma_loss: 0.0026 - mu_loss: 0.0175 - jump_sigma_loss: 0.0030 - jump_mu_loss: 0.0090 - lambda_loss: 6.6771e-05 - sigma_mean_absolute_percentage_error: 158.1034 - sigma_r2: 0.2157 - mu_mean_absolute_percentage_error: 266.4196 - mu_r2: 0.7900 - jump_sigma_mean_absolute_percentage_error: 182.5469 - jump_sigma_r2: 0.0875 - jump_mu_mean_absolute_percentage_error: 91.8385 - jump_mu_r2: 0.8915 - lambda_mean_absolute_percentage_error: 132.3332 - lambda_r2: -0.2917    
SAVE
[master cf49ed9] updated history, predictions and model
 4 files changed, 6 insertions(+), 22 deletions(-)
Branch master set up to track remote branch master from origin.
Epoch 1/1
10/10 [==============================] - 124s - loss: 0.0310 - sigma_loss: 0.0026 - mu_loss: 0.0163 - jump_sigma_loss: 0.0030 - jump_mu_loss: 0.0091 - lambda_loss: 6.1937e-05 - sigma_mean_absolute_percentage_error: 157.1786 - sigma_r2: 0.2261 - mu_mean_absolute_percentage_er

KeyboardInterrupt: 

In [ ]:
print()

# LESS SIMPLE Covnet Multiple Output with ELUs 10 Layers

In [14]:
import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, LSTM, MaxPooling2D, Input, merge
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras import backend as K

In [15]:
def r2(y_true, y_pred):
    """
    returns the correlation coefficient of y_pred against y_true.

    :param y_true: the true values (independent variable)
    :param y_pred: the predicted values (dependent variable)
    """
    
    SSE = K.sum(K.square(y_true-y_pred))
    SST = K.sum(K.square(y_true-K.mean(y_true)))
    
    return 1-SSE/SST

In [22]:
convnet_mo_elu = load_model("FULLMODEL_6_hidden_LAYERS.h5", custom_objects={"r2":r2})

IOError: Unable to open file (unable to open file: name = 'FULLMODEL_6_hidden_LAYERS.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [41]:
#covnet_mo_elu = load_model("FULLMODEL_10LAYERS_COVNET.h5", custom_objects={"r2":r2})

input_1 = Input(shape = (40, 50, 1))

layer1 = Conv2D(32, (3, 3), activation='elu')(input_1)
layer2 = Conv2D(32, (3, 3), activation='elu')(layer1)
layer3 = MaxPooling2D(pool_size=(2,2))(layer2)

layer4 = Conv2D(64, (3, 3), activation='elu')(layer3)
layer5 = Conv2D(64, (3, 3), activation='elu')(layer4)
layer6 = MaxPooling2D(pool_size=(2,2))(layer5)

layer4 = Conv2D(128, (3, 3), activation='elu')(layer3)
layer5 = Conv2D(128, (3, 3), activation='elu')(layer4)
layer6 = MaxPooling2D(pool_size=(2,2))(layer5)

flatten = Flatten()(layer6)

output1_layer1 = Dense(32, activation='elu')(flatten)
output1_layer2 = Dense(16, activation='elu')(output1_layer1)
output1 = Dense(1, name="sigma")(output1_layer2)

output2_layer1 = Dense(32, activation='elu')(flatten)
output2_layer2 = Dense(16, activation='elu')(output2_layer1)
output2 = Dense(1, name="mu")(output2_layer2)

output3_layer1 = Dense(32, activation='elu')(flatten)
output3_layer2 = Dense(16, activation='elu')(output3_layer1)
output3 = Dense(1, name="jump_sigma")(output3_layer2)

output4_layer1 = Dense(32, activation='elu')(flatten)
output4_layer2 = Dense(16, activation='elu')(output4_layer1)
output4 = Dense(1, name="jump_mu")(output4_layer2)

output5_layer1 = Dense(32, activation='elu')(flatten)
output5_layer2 = Dense(16, activation='elu')(output5_layer1)
output5 = Dense(1, name="lambda")(output5_layer2)

convnet_mo_elu = Model(input = input_1, output=[output1, output2, output3, output4, output5])
convnet_mo_elu.compile('adam', 'mean_squared_error', metrics=['mean_absolute_percentage_error', r2])

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [20]:
#covnet_mo_elu = covnet_multiple_ELUs_10_layers()
#covnet_mo_elu.load_weights("RELAXED_ELU_COVNET.h5")

In [21]:
#covnet_mo_elu.save(H5_DIR_COVNET_MULTIPLE_ELU)

In [17]:
test_set_pred = standard_convolutional_test_set()

0
100
200
300
400
500
600
700
800
900


In [52]:
train_gen = convolutional_batch_generator(paramset_size=1750, paths_p_paramset=1, constraint_gbm_mu=(-0.5, 0.5))

In [42]:
histories_elu = []#np.load("SIMPLE_histories_elu_6_hidden_LAYERS.npy").tolist()
covnet_mo_elu_predictions = []# np.load("SIMPLE_mo_elu_predictions_6_hidden_LAYERS.npy").tolist()

In [39]:
smallest_loss = 20

In [ ]:
for i in range(150000):
    history = convnet_mo_elu.fit_generator(train_gen, samples_per_epoch = 10, nb_epoch = 1)
    histories_elu.append(history.history)

    predictions_elu = convnet_mo_elu.predict(test_set_pred)

    covnet_mo_elu_predictions.append(predictions_elu)
    
    if history.history['loss'][-1]<smallest_loss:
        print("SAVE")
        smallest_loss = history.history['loss'][-1]
        convnet_mo_elu.save("LESS_SIMPLE_6_hidden_LAYERS.h5")
    
    if i%5 == 0:
        np.save("LESS_SIMPLE_histories_elu_6_hidden_LAYERS.npy", histories_elu)
        np.save("LESS_SIMPLE_mo_elu_predictions_6_hidden_LAYERS.npy", covnet_mo_elu_predictions)

        !git config --global user.email "rwmmostert@gmail.com"
        !git config --global user.name "RWMostert"
        !git add --all
        !git commit -am "updated history, predictions and model"
        !git push -u origin master --quiet

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=1, steps_per_epoch=10)`
  


Epoch 1/1
10/10 [==============================] - 135s - loss: 0.0139 - sigma_loss: 1.2166e-04 - mu_loss: 0.0035 - jump_sigma_loss: 0.0029 - jump_mu_loss: 0.0073 - lambda_loss: 4.5955e-05 - sigma_mean_absolute_percentage_error: 21.1051 - sigma_r2: 0.9632 - mu_mean_absolute_percentage_error: 101.7523 - mu_r2: 0.9584 - jump_sigma_mean_absolute_percentage_error: 178.6135 - jump_sigma_r2: 0.1273 - jump_mu_mean_absolute_percentage_error: 132.7918 - jump_mu_r2: 0.9131 - lambda_mean_absolute_percentage_error: 105.5221 - lambda_r2: 0.1059    
SAVE
[master bc54140] updated history, predictions and model
 4 files changed, 3 insertions(+), 3 deletions(-)
Branch master set up to track remote branch master from origin.
Epoch 1/1
10/10 [==============================] - 136s - loss: 0.0135 - sigma_loss: 9.8040e-05 - mu_loss: 0.0034 - jump_sigma_loss: 0.0029 - jump_mu_loss: 0.0071 - lambda_loss: 4.6092e-05 - sigma_mean_absolute_percentage_error: 17.5162 - sigma_r2: 0.9702 - mu_mean_absolute_percenta

10/10 [==============================] - 137s - loss: 0.0125 - sigma_loss: 8.0327e-05 - mu_loss: 0.0033 - jump_sigma_loss: 0.0026 - jump_mu_loss: 0.0064 - lambda_loss: 3.9214e-05 - sigma_mean_absolute_percentage_error: 16.8791 - sigma_r2: 0.9753 - mu_mean_absolute_percentage_error: 409.3439 - mu_r2: 0.9602 - jump_sigma_mean_absolute_percentage_error: 165.6989 - jump_sigma_r2: 0.1889 - jump_mu_mean_absolute_percentage_error: 80.4612 - jump_mu_r2: 0.9234 - lambda_mean_absolute_percentage_error: 92.2146 - lambda_r2: 0.2359    
SAVE
Epoch 1/1
10/10 [==============================] - 136s - loss: 0.0125 - sigma_loss: 8.6204e-05 - mu_loss: 0.0032 - jump_sigma_loss: 0.0027 - jump_mu_loss: 0.0065 - lambda_loss: 3.8615e-05 - sigma_mean_absolute_percentage_error: 17.9318 - sigma_r2: 0.9738 - mu_mean_absolute_percentage_error: 151.2248 - mu_r2: 0.9615 - jump_sigma_mean_absolute_percentage_error: 167.1943 - jump_sigma_r2: 0.1918 - jump_mu_mean_absolute_percentage_error: 421.5596 - jump_mu_r2: 0.92

10/10 [==============================] - 137s - loss: 0.0127 - sigma_loss: 8.4577e-05 - mu_loss: 0.0038 - jump_sigma_loss: 0.0026 - jump_mu_loss: 0.0062 - lambda_loss: 4.5858e-05 - sigma_mean_absolute_percentage_error: 18.5231 - sigma_r2: 0.9744 - mu_mean_absolute_percentage_error: 136.9708 - mu_r2: 0.9545 - jump_sigma_mean_absolute_percentage_error: 165.0148 - jump_sigma_r2: 0.2197 - jump_mu_mean_absolute_percentage_error: 120.4236 - jump_mu_r2: 0.9262 - lambda_mean_absolute_percentage_error: 94.1431 - lambda_r2: 0.1085    
Epoch 1/1
10/10 [==============================] - 136s - loss: 0.0127 - sigma_loss: 7.9031e-05 - mu_loss: 0.0038 - jump_sigma_loss: 0.0026 - jump_mu_loss: 0.0061 - lambda_loss: 3.5915e-05 - sigma_mean_absolute_percentage_error: 17.6332 - sigma_r2: 0.9760 - mu_mean_absolute_percentage_error: 142.7013 - mu_r2: 0.9536 - jump_sigma_mean_absolute_percentage_error: 158.3913 - jump_sigma_r2: 0.2162 - jump_mu_mean_absolute_percentage_error: 133.4705 - jump_mu_r2: 0.9264 -

10/10 [==============================] - 137s - loss: 0.0117 - sigma_loss: 6.8273e-05 - mu_loss: 0.0033 - jump_sigma_loss: 0.0025 - jump_mu_loss: 0.0058 - lambda_loss: 3.1128e-05 - sigma_mean_absolute_percentage_error: 16.3624 - sigma_r2: 0.9793 - mu_mean_absolute_percentage_error: 70.9165 - mu_r2: 0.9602 - jump_sigma_mean_absolute_percentage_error: 166.1997 - jump_sigma_r2: 0.2420 - jump_mu_mean_absolute_percentage_error: 107.2483 - jump_mu_r2: 0.9307 - lambda_mean_absolute_percentage_error: 89.4078 - lambda_r2: 0.3955    
Epoch 1/1
10/10 [==============================] - 136s - loss: 0.0117 - sigma_loss: 6.7284e-05 - mu_loss: 0.0033 - jump_sigma_loss: 0.0026 - jump_mu_loss: 0.0058 - lambda_loss: 2.8802e-05 - sigma_mean_absolute_percentage_error: 15.1822 - sigma_r2: 0.9798 - mu_mean_absolute_percentage_error: 109.0811 - mu_r2: 0.9603 - jump_sigma_mean_absolute_percentage_error: 161.9186 - jump_sigma_r2: 0.2309 - jump_mu_mean_absolute_percentage_error: 80.6473 - jump_mu_r2: 0.9304 - l

...

10/10 [==============================] - 134s - loss: 0.0099 - sigma_loss: 4.7860e-05 - mu_loss: 0.0029 - jump_sigma_loss: 0.0022 - jump_mu_loss: 0.0048 - lambda_loss: 1.6951e-05 - sigma_mean_absolute_percentage_error: 14.2141 - sigma_r2: 0.9855 - mu_mean_absolute_percentage_error: 98.8681 - mu_r2: 0.9657 - jump_sigma_mean_absolute_percentage_error: 150.0660 - jump_sigma_r2: 0.3365 - jump_mu_mean_absolute_percentage_error: 81.1194 - jump_mu_r2: 0.9428 - lambda_mean_absolute_percentage_error: 67.0755 - lambda_r2: 0.6710     
Epoch 1/1
10/10 [==============================] - 134s - loss: 0.0104 - sigma_loss: 5.1475e-05 - mu_loss: 0.0033 - jump_sigma_loss: 0.0023 - jump_mu_loss: 0.0048 - lambda_loss: 1.6886e-05 - sigma_mean_absolute_percentage_error: 15.1213 - sigma_r2: 0.9844 - mu_mean_absolute_percentage_error: 130.1235 - mu_r2: 0.9604 - jump_sigma_mean_absolute_percentage_error: 154.0146 - jump_sigma_r2: 0.3190 - jump_mu_mean_absolute_percentage_error: 75.2344 - jump_mu_r2: 0.9436 - l

10/10 [==============================] - 139s - loss: 0.0094 - sigma_loss: 4.1829e-05 - mu_loss: 0.0026 - jump_sigma_loss: 0.0022 - jump_mu_loss: 0.0045 - lambda_loss: 1.6614e-05 - sigma_mean_absolute_percentage_error: 11.4274 - sigma_r2: 0.9873 - mu_mean_absolute_percentage_error: 137.1119 - mu_r2: 0.9687 - jump_sigma_mean_absolute_percentage_error: 150.9117 - jump_sigma_r2: 0.3463 - jump_mu_mean_absolute_percentage_error: 70.0369 - jump_mu_r2: 0.9459 - lambda_mean_absolute_percentage_error: 65.3543 - lambda_r2: 0.6756   
Epoch 1/1
10/10 [==============================] - 140s - loss: 0.0096 - sigma_loss: 4.6970e-05 - mu_loss: 0.0027 - jump_sigma_loss: 0.0022 - jump_mu_loss: 0.0046 - lambda_loss: 1.9681e-05 - sigma_mean_absolute_percentage_error: 13.3570 - sigma_r2: 0.9857 - mu_mean_absolute_percentage_error: 80.4339 - mu_r2: 0.9674 - jump_sigma_mean_absolute_percentage_error: 150.8795 - jump_sigma_r2: 0.3338 - jump_mu_mean_absolute_percentage_error: 76.9652 - jump_mu_r2: 0.9438 - lam

10/10 [==============================] - 136s - loss: 0.0100 - sigma_loss: 5.2197e-05 - mu_loss: 0.0030 - jump_sigma_loss: 0.0022 - jump_mu_loss: 0.0047 - lambda_loss: 1.8248e-05 - sigma_mean_absolute_percentage_error: 14.0231 - sigma_r2: 0.9844 - mu_mean_absolute_percentage_error: 79.6383 - mu_r2: 0.9635 - jump_sigma_mean_absolute_percentage_error: 144.5294 - jump_sigma_r2: 0.3352 - jump_mu_mean_absolute_percentage_error: 99.2434 - jump_mu_r2: 0.9439 - lambda_mean_absolute_percentage_error: 63.7759 - lambda_r2: 0.6501     
[master 2519e63] updated history, predictions and model
 3 files changed, 20 insertions(+), 1 deletion(-)
Branch master set up to track remote branch master from origin.
Epoch 1/1
10/10 [==============================] - 137s - loss: 0.0096 - sigma_loss: 4.9125e-05 - mu_loss: 0.0029 - jump_sigma_loss: 0.0021 - jump_mu_loss: 0.0045 - lambda_loss: 1.6638e-05 - sigma_mean_absolute_percentage_error: 13.8831 - sigma_r2: 0.9849 - mu_mean_absolute_percentage_error: 81.4313

10/10 [==============================] - 138s - loss: 0.0096 - sigma_loss: 4.7600e-05 - mu_loss: 0.0029 - jump_sigma_loss: 0.0021 - jump_mu_loss: 0.0046 - lambda_loss: 1.8247e-05 - sigma_mean_absolute_percentage_error: 12.4882 - sigma_r2: 0.9856 - mu_mean_absolute_percentage_error: 167.5444 - mu_r2: 0.9650 - jump_sigma_mean_absolute_percentage_error: 143.4117 - jump_sigma_r2: 0.3657 - jump_mu_mean_absolute_percentage_error: 358.7601 - jump_mu_r2: 0.9444 - lambda_mean_absolute_percentage_error: 68.3353 - lambda_r2: 0.6447    
Epoch 1/1
10/10 [==============================] - 135s - loss: 0.0095 - sigma_loss: 4.4022e-05 - mu_loss: 0.0027 - jump_sigma_loss: 0.0022 - jump_mu_loss: 0.0046 - lambda_loss: 1.6938e-05 - sigma_mean_absolute_percentage_error: 11.7574 - sigma_r2: 0.9866 - mu_mean_absolute_percentage_error: 67.3671 - mu_r2: 0.9670 - jump_sigma_mean_absolute_percentage_error: 150.2380 - jump_sigma_r2: 0.3419 - jump_mu_mean_absolute_percentage_error: 78.9808 - jump_mu_r2: 0.9456 - l

10/10 [==============================] - 137s - loss: 0.0095 - sigma_loss: 4.8098e-05 - mu_loss: 0.0030 - jump_sigma_loss: 0.0021 - jump_mu_loss: 0.0044 - lambda_loss: 1.5600e-05 - sigma_mean_absolute_percentage_error: 12.1148 - sigma_r2: 0.9855 - mu_mean_absolute_percentage_error: 69.5714 - mu_r2: 0.9646 - jump_sigma_mean_absolute_percentage_error: 138.7301 - jump_sigma_r2: 0.3669 - jump_mu_mean_absolute_percentage_error: 98.7823 - jump_mu_r2: 0.9474 - lambda_mean_absolute_percentage_error: 63.5168 - lambda_r2: 0.6986    
Epoch 1/1
10/10 [==============================] - 139s - loss: 0.0097 - sigma_loss: 4.8907e-05 - mu_loss: 0.0032 - jump_sigma_loss: 0.0021 - jump_mu_loss: 0.0044 - lambda_loss: 1.5404e-05 - sigma_mean_absolute_percentage_error: 13.6479 - sigma_r2: 0.9851 - mu_mean_absolute_percentage_error: 96.4895 - mu_r2: 0.9612 - jump_sigma_mean_absolute_percentage_error: 137.0339 - jump_sigma_r2: 0.3765 - jump_mu_mean_absolute_percentage_error: 59.2626 - jump_mu_r2: 0.9468 - lam

# THIS HERE DILUTED Covnet Multiple Output with ELUs 10 Layers

In [9]:
from keras.layers.merge import Concatenate

In [9]:
import tensorflow as tf
import keras
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Flatten, LSTM, MaxPooling2D, Input, merge
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras import backend as K

In [10]:
def r2(y_true, y_pred):
    """
    returns the correlation coefficient of y_pred against y_true.

    :param y_true: the true values (independent variable)
    :param y_pred: the predicted values (dependent variable)
    """
    
    SSE = K.sum(K.square(y_true-y_pred))
    SST = K.sum(K.square(y_true-K.mean(y_true)))
    
    return 1-SSE/SST

In [11]:
import h5py as h5py

In [26]:
!sudo apt-get install libhdf5-dev

sudo: apt-get: command not found


In [12]:
convnet_mo_elu = load_model("diluted_FULLMODEL_6_hidden_LAYERS.h5", custom_objects={"r2":r2})

/usr/local/lib64/python2.7/site-packages/keras/engine/topology.py:1252: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [36]:
input_1 = Input(shape = (40, 50, 1))

layer1 = Conv2D(32, (6, 6), activation='elu')(input_1)
layer2 = Conv2D(32, (6, 6), activation='elu')(layer1)
layer3 = MaxPooling2D(pool_size=(2,2))(layer2)

dil_layer1 = Conv2D(32, (3, 3), dilation_rate=(3, 3), activation='elu')(input_1)
dil_layer2 = Conv2D(32, (3, 3), dilation_rate=(2, 2), activation='elu')(dil_layer1)
dil_layer3 = MaxPooling2D(pool_size=(2,2))(dil_layer2)

concat = merge([layer3, dil_layer3], mode="concat")

layer4 = Conv2D(64, (3, 3), activation='elu')(concat)
layer5 = Conv2D(64, (3, 3), activation='elu')(layer4)
layer6 = MaxPooling2D(pool_size=(2,2))(layer5)

flatten = Flatten()(layer6)

output1_layer1 = Dense(32, activation='elu')(flatten)
output1_layer2 = Dense(16, activation='elu')(output1_layer1)
output1 = Dense(1, name="sigma")(output1_layer2)

output2_layer1 = Dense(32, activation='elu')(flatten)
output2_layer2 = Dense(16, activation='elu')(output2_layer1)
output2 = Dense(1, name="mu")(output2_layer2)

output3_layer1 = Dense(32, activation='elu')(flatten)
output3_layer2 = Dense(16, activation='elu')(output3_layer1)
output3 = Dense(1, name="jump_sigma")(output3_layer2)

output4_layer1 = Dense(32, activation='elu')(flatten)
output4_layer2 = Dense(16, activation='elu')(output4_layer1)
output4 = Dense(1, name="jump_mu")(output4_layer2)

output5_layer1 = Dense(32, activation='elu')(flatten)
output5_layer2 = Dense(16, activation='elu')(output5_layer1)
output5 = Dense(1, name="lambda")(output5_layer2)

convnet_mo_elu = Model(input = input_1, output=[output1, output2, output3, output4, output5])
convnet_mo_elu.compile('adam', 'mean_squared_error', metrics=['mean_absolute_percentage_error', r2])

/usr/lib/python3.4/dist-packages/ipykernel_launcher.py:11: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib64/python3.4/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/lib/python3.4/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [20]:
#covnet_mo_elu = covnet_multiple_ELUs_10_layers()
#covnet_mo_elu.load_weights("RELAXED_ELU_COVNET.h5")

In [21]:
#covnet_mo_elu.save(H5_DIR_COVNET_MULTIPLE_ELU)

In [13]:
test_set_pred = standard_convolutional_test_set()

0
100
200
300
400
500
600
700
800
900


In [20]:
train_gen = convolutional_batch_generator(paramset_size=500, paths_p_paramset=3, constraint_gbm_mu=(-0.5, 0.5))

In [15]:
histories_elu = np.load("diluted_covnet_histories_elu_6_hidden_LAYERS.npy").tolist()
covnet_mo_elu_predictions = np.load("diluted_covnet_mo_elu_predictions_6_hidden_LAYERS.npy").tolist()

In [16]:
smallest_loss = 20

In [21]:
for i in range(150000):
    history = convnet_mo_elu.fit_generator(train_gen, samples_per_epoch = 10, nb_epoch = 1)
    histories_elu.append(history.history)

    predictions_elu = convnet_mo_elu.predict(test_set_pred)

    covnet_mo_elu_predictions.append(predictions_elu)
    
    if history.history['loss'][-1]<smallest_loss:
        print("SAVE")
        smallest_loss = history.history['loss'][-1]
        convnet_mo_elu.save("diluted_FULLMODEL_6_hidden_LAYERS.h5")
    
    if i%5 == 0:
        np.save("diluted_covnet_histories_elu_6_hidden_LAYERS.npy", histories_elu)
        np.save("diluted_covnet_mo_elu_predictions_6_hidden_LAYERS.npy", covnet_mo_elu_predictions)

        !git config --global user.email "rwmmostert@gmail.com"
        !git config --global user.name "RWMostert"
        !git add --all
        !git commit -am "updated history, predictions and model"
        !git push -u origin master --quiet

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=1, steps_per_epoch=10)`
  


Epoch 1/1
10/10 [==============================] - 115s - loss: 0.0127 - sigma_loss: 1.3061e-04 - mu_loss: 0.0039 - jump_sigma_loss: 0.0025 - jump_mu_loss: 0.0062 - lambda_loss: 1.9385e-05 - sigma_mean_absolute_percentage_error: 23.1099 - sigma_r2: 0.9602 - mu_mean_absolute_percentage_error: 119.5370 - mu_r2: 0.9530 - jump_sigma_mean_absolute_percentage_error: 158.5887 - jump_sigma_r2: 0.2509 - jump_mu_mean_absolute_percentage_error: 121.7904 - jump_mu_r2: 0.9274 - lambda_mean_absolute_percentage_error: 82.0272 - lambda_r2: 0.6193    
SAVE
[master 2d259dc] updated history, predictions and model
 4 files changed, 32 insertions(+), 27 deletions(-)
Branch master set up to track remote branch master from origin.
Epoch 1/1
10/10 [==============================] - 118s - loss: 0.0126 - sigma_loss: 1.2259e-04 - mu_loss: 0.0040 - jump_sigma_loss: 0.0024 - jump_mu_loss: 0.0060 - lambda_loss: 2.0020e-05 - sigma_mean_absolute_percentage_error: 23.9235 - sigma_r2: 0.9627 - mu_mean_absolute_percent

10/10 [==============================] - 118s - loss: 0.0128 - sigma_loss: 1.1203e-04 - mu_loss: 0.0043 - jump_sigma_loss: 0.0024 - jump_mu_loss: 0.0060 - lambda_loss: 2.0403e-05 - sigma_mean_absolute_percentage_error: 24.3946 - sigma_r2: 0.9662 - mu_mean_absolute_percentage_error: 109.4488 - mu_r2: 0.9477 - jump_sigma_mean_absolute_percentage_error: 160.1737 - jump_sigma_r2: 0.2694 - jump_mu_mean_absolute_percentage_error: 69.9613 - jump_mu_r2: 0.9272 - lambda_mean_absolute_percentage_error: 77.5324 - lambda_r2: 0.5861    
Epoch 1/1
10/10 [==============================] - 119s - loss: 0.0123 - sigma_loss: 1.2022e-04 - mu_loss: 0.0042 - jump_sigma_loss: 0.0023 - jump_mu_loss: 0.0057 - lambda_loss: 2.0690e-05 - sigma_mean_absolute_percentage_error: 19.6040 - sigma_r2: 0.9624 - mu_mean_absolute_percentage_error: 157.5255 - mu_r2: 0.9505 - jump_sigma_mean_absolute_percentage_error: 146.7098 - jump_sigma_r2: 0.2908 - jump_mu_mean_absolute_percentage_error: 74.7728 - jump_mu_r2: 0.9313 - l

10/10 [==============================] - 114s - loss: 0.0124 - sigma_loss: 1.2926e-04 - mu_loss: 0.0037 - jump_sigma_loss: 0.0024 - jump_mu_loss: 0.0061 - lambda_loss: 2.0476e-05 - sigma_mean_absolute_percentage_error: 24.4112 - sigma_r2: 0.9605 - mu_mean_absolute_percentage_error: 163.8629 - mu_r2: 0.9554 - jump_sigma_mean_absolute_percentage_error: 149.0956 - jump_sigma_r2: 0.2656 - jump_mu_mean_absolute_percentage_error: 107.0388 - jump_mu_r2: 0.9261 - lambda_mean_absolute_percentage_error: 81.2071 - lambda_r2: 0.6046    
Epoch 1/1
10/10 [==============================] - 114s - loss: 0.0120 - sigma_loss: 1.2417e-04 - mu_loss: 0.0039 - jump_sigma_loss: 0.0024 - jump_mu_loss: 0.0056 - lambda_loss: 1.8181e-05 - sigma_mean_absolute_percentage_error: 25.2890 - sigma_r2: 0.9620 - mu_mean_absolute_percentage_error: 74.1834 - mu_r2: 0.9529 - jump_sigma_mean_absolute_percentage_error: 155.4009 - jump_sigma_r2: 0.2775 - jump_mu_mean_absolute_percentage_error: 88.3188 - jump_mu_r2: 0.9321 - l

10/10 [==============================] - 117s - loss: 0.0124 - sigma_loss: 9.9145e-05 - mu_loss: 0.0040 - jump_sigma_loss: 0.0025 - jump_mu_loss: 0.0059 - lambda_loss: 1.8943e-05 - sigma_mean_absolute_percentage_error: 19.0118 - sigma_r2: 0.9696 - mu_mean_absolute_percentage_error: 56.0474 - mu_r2: 0.9524 - jump_sigma_mean_absolute_percentage_error: 166.5304 - jump_sigma_r2: 0.2583 - jump_mu_mean_absolute_percentage_error: 78.1880 - jump_mu_r2: 0.9298 - lambda_mean_absolute_percentage_error: 82.2507 - lambda_r2: 0.6370    
Epoch 1/1
10/10 [==============================] - 117s - loss: 0.0116 - sigma_loss: 9.7731e-05 - mu_loss: 0.0037 - jump_sigma_loss: 0.0024 - jump_mu_loss: 0.0054 - lambda_loss: 2.0592e-05 - sigma_mean_absolute_percentage_error: 19.9704 - sigma_r2: 0.9702 - mu_mean_absolute_percentage_error: 77.1965 - mu_r2: 0.9539 - jump_sigma_mean_absolute_percentage_error: 160.6262 - jump_sigma_r2: 0.2749 - jump_mu_mean_absolute_percentage_error: 475.6020 - jump_mu_r2: 0.9365 - la

...

10/10 [==============================] - 115s - loss: 0.0115 - sigma_loss: 9.4143e-05 - mu_loss: 0.0038 - jump_sigma_loss: 0.0023 - jump_mu_loss: 0.0053 - lambda_loss: 1.8673e-05 - sigma_mean_absolute_percentage_error: 19.4352 - sigma_r2: 0.9717 - mu_mean_absolute_percentage_error: 75.8911 - mu_r2: 0.9536 - jump_sigma_mean_absolute_percentage_error: 155.8872 - jump_sigma_r2: 0.2937 - jump_mu_mean_absolute_percentage_error: 62.6205 - jump_mu_r2: 0.9372 - lambda_mean_absolute_percentage_error: 73.9602 - lambda_r2: 0.6306    
[master 531506f] updated history, predictions and model
 3 files changed, 22 insertions(+), 1 deletion(-)
Branch master set up to track remote branch master from origin.
Epoch 1/1
10/10 [==============================] - 119s - loss: 0.0116 - sigma_loss: 9.2619e-05 - mu_loss: 0.0040 - jump_sigma_loss: 0.0022 - jump_mu_loss: 0.0053 - lambda_loss: 1.6620e-05 - sigma_mean_absolute_percentage_error: 19.6951 - sigma_r2: 0.9719 - mu_mean_absolute_percentage_error: 134.3866

10/10 [==============================] - 117s - loss: 0.0115 - sigma_loss: 1.0605e-04 - mu_loss: 0.0037 - jump_sigma_loss: 0.0023 - jump_mu_loss: 0.0054 - lambda_loss: 1.7917e-05 - sigma_mean_absolute_percentage_error: 21.6307 - sigma_r2: 0.9681 - mu_mean_absolute_percentage_error: 90.3119 - mu_r2: 0.9561 - jump_sigma_mean_absolute_percentage_error: 151.3977 - jump_sigma_r2: 0.3100 - jump_mu_mean_absolute_percentage_error: 228.3776 - jump_mu_r2: 0.9345 - lambda_mean_absolute_percentage_error: 76.1037 - lambda_r2: 0.6592    
Epoch 1/1
10/10 [==============================] - 117s - loss: 0.0116 - sigma_loss: 9.0649e-05 - mu_loss: 0.0042 - jump_sigma_loss: 0.0023 - jump_mu_loss: 0.0050 - lambda_loss: 1.6922e-05 - sigma_mean_absolute_percentage_error: 21.3873 - sigma_r2: 0.9728 - mu_mean_absolute_percentage_error: 117.9198 - mu_r2: 0.9502 - jump_sigma_mean_absolute_percentage_error: 148.3024 - jump_sigma_r2: 0.3091 - jump_mu_mean_absolute_percentage_error: 67.1838 - jump_mu_r2: 0.9403 - l

10/10 [==============================] - 116s - loss: 0.0109 - sigma_loss: 9.6034e-05 - mu_loss: 0.0036 - jump_sigma_loss: 0.0023 - jump_mu_loss: 0.0049 - lambda_loss: 1.7240e-05 - sigma_mean_absolute_percentage_error: 18.2555 - sigma_r2: 0.9705 - mu_mean_absolute_percentage_error: 72.0852 - mu_r2: 0.9562 - jump_sigma_mean_absolute_percentage_error: 149.1527 - jump_sigma_r2: 0.3164 - jump_mu_mean_absolute_percentage_error: 84.0412 - jump_mu_r2: 0.9403 - lambda_mean_absolute_percentage_error: 68.5058 - lambda_r2: 0.6663    
SAVE
Epoch 1/1
10/10 [==============================] - 116s - loss: 0.0115 - sigma_loss: 8.7889e-05 - mu_loss: 0.0038 - jump_sigma_loss: 0.0023 - jump_mu_loss: 0.0053 - lambda_loss: 1.7894e-05 - sigma_mean_absolute_percentage_error: 19.2195 - sigma_r2: 0.9729 - mu_mean_absolute_percentage_error: 99.1275 - mu_r2: 0.9536 - jump_sigma_mean_absolute_percentage_error: 155.5600 - jump_sigma_r2: 0.3071 - jump_mu_mean_absolute_percentage_error: 111.2755 - jump_mu_r2: 0.9372

10/10 [==============================] - 118s - loss: 0.0108 - sigma_loss: 8.2989e-05 - mu_loss: 0.0035 - jump_sigma_loss: 0.0021 - jump_mu_loss: 0.0050 - lambda_loss: 1.6689e-05 - sigma_mean_absolute_percentage_error: 19.7334 - sigma_r2: 0.9752 - mu_mean_absolute_percentage_error: 110.7421 - mu_r2: 0.9572 - jump_sigma_mean_absolute_percentage_error: 134.4055 - jump_sigma_r2: 0.3330 - jump_mu_mean_absolute_percentage_error: 73.6481 - jump_mu_r2: 0.9393 - lambda_mean_absolute_percentage_error: 63.9819 - lambda_r2: 0.6722    
Epoch 1/1
10/10 [==============================] - 118s - loss: 0.0103 - sigma_loss: 8.3402e-05 - mu_loss: 0.0034 - jump_sigma_loss: 0.0022 - jump_mu_loss: 0.0046 - lambda_loss: 1.7922e-05 - sigma_mean_absolute_percentage_error: 18.0304 - sigma_r2: 0.9746 - mu_mean_absolute_percentage_error: 144.5337 - mu_r2: 0.9591 - jump_sigma_mean_absolute_percentage_error: 154.4393 - jump_sigma_r2: 0.3144 - jump_mu_mean_absolute_percentage_error: 66.1370 - jump_mu_r2: 0.9446 - l

KeyboardInterrupt: 

# SAME Covnet Multiple Output with ELUs 10 Layers

In [41]:
from keras.layers.merge import Concatenate

In [35]:
import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, LSTM, MaxPooling2D, Input, Merge
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras import backend as K

In [36]:
def r2(y_true, y_pred):
    """
    returns the correlation coefficient of y_pred against y_true.

    :param y_true: the true values (independent variable)
    :param y_pred: the predicted values (dependent variable)
    """
    
    SSE = K.sum(K.square(y_true-y_pred))
    SST = K.sum(K.square(y_true-K.mean(y_true)))
    
    return 1-SSE/SST

In [40]:
#covnet_mo_elu = load_model("FULLMODEL_10LAYERS_COVNET.h5", custom_objects={"r2":r2})

input_1 = Input(shape = (40, 50, 1))

layer1 = Conv2D(128, (6, 6), activation='elu')(input_1)
layer2 = Conv2D(64, (6, 6), activation='elu')(layer1)
layer3 = MaxPooling2D(pool_size=(2,2))(layer2)

dil_layer1 = Conv2D(128, (3, 3), dilation_rate=(3, 3), activation='elu')(input_1)
dil_layer2 = Conv2D(64, (3, 3), dilation_rate=(2, 2), activation='elu')(dil_layer1)
dil_layer3 = MaxPooling2D(pool_size=(2,2))(dil_layer2)

concat = merge([layer3, dil_layer3], mode="concat")

layer4 = Conv2D(64, (3, 3), activation='elu')(concat)
layer5 = Conv2D(32, (3, 3), activation='elu')(layer4)
layer6 = MaxPooling2D(pool_size=(2,2))(layer5)

flatten = Flatten()(layer6)

output1_layer1 = Dense(32, activation='elu')(flatten)
output1_layer2 = Dense(16, activation='elu')(output1_layer1)
output1 = Dense(1, name="sigma")(output1_layer2)

output2_layer1 = Dense(32, activation='elu')(flatten)
output2_layer2 = Dense(16, activation='elu')(output2_layer1)
output2 = Dense(1, name="mu")(output2_layer2)

output3_layer1 = Dense(32, activation='elu')(flatten)
output3_layer2 = Dense(16, activation='elu')(output3_layer1)
output3 = Dense(1, name="jump_sigma")(output3_layer2)

output4_layer1 = Dense(32, activation='elu')(flatten)
output4_layer2 = Dense(16, activation='elu')(output4_layer1)
output4 = Dense(1, name="jump_mu")(output4_layer2)

output5_layer1 = Dense(32, activation='elu')(flatten)
output5_layer2 = Dense(16, activation='elu')(output5_layer1)
output5 = Dense(1, name="lambda")(output5_layer2)

convnet_mo_elu = Model(input = input_1, output=[output1, output2, output3, output4, output5])
convnet_mo_elu.compile('adam', 'mean_squared_error', metrics=['mean_absolute_percentage_error', r2])

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [20]:
#covnet_mo_elu = covnet_multiple_ELUs_10_layers()
#covnet_mo_elu.load_weights("RELAXED_ELU_COVNET.h5")

In [21]:
#covnet_mo_elu.save(H5_DIR_COVNET_MULTIPLE_ELU)

In [17]:
test_set_pred = standard_convolutional_test_set()

0
100
200
300
400
500
600
700
800
900


In [45]:
train_gen = convolutional_batch_generator(paramset_size=500, paths_p_paramset=1, constraint_gbm_mu=(-0.5, 0.5))

In [42]:
histories_elu = []#np.load("covnet_histories_elu_10LAYERS.npy").tolist()
covnet_mo_elu_predictions = []# np.load("covnet_mo_elu_predictions_10LAYERS.npy").tolist()

In [43]:
smallest_loss = 20

In [46]:
for i in range(150000):
    history = convnet_mo_elu.fit_generator(train_gen, samples_per_epoch = 10, nb_epoch = 1)
    histories_elu.append(history.history)

    predictions_elu = convnet_mo_elu.predict(test_set_pred)

    covnet_mo_elu_predictions.append(predictions_elu)
    
    if history.history['loss'][-1]<smallest_loss:
        print("SAVE")
        smallest_loss = history.history['loss'][-1]
        convnet_mo_elu.save("beta_FULLMODEL_6_hidden_LAYERS.h5")
    
    if i%5 == 0:
        np.save("beta_covnet_histories_elu_6_hidden_LAYERS.npy", histories_elu)
        np.save("beta_covnet_mo_elu_predictions_6_hidden_LAYERS.npy", covnet_mo_elu_predictions)

        !git config --global user.email "rwmmostert@gmail.com"
        !git config --global user.name "RWMostert"
        !git add --all
        !git commit -am "updated history, predictions and model"
        !git push -u origin master --quiet

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=1, steps_per_epoch=10)`
  


Epoch 1/1
10/10 [==============================] - 39s - loss: 0.0294 - sigma_loss: 9.7586e-04 - mu_loss: 0.0141 - jump_sigma_loss: 0.0033 - jump_mu_loss: 0.0110 - lambda_loss: 4.6384e-05 - sigma_mean_absolute_percentage_error: 82.0049 - sigma_r2: 0.7011 - mu_mean_absolute_percentage_error: 10069.2652 - mu_r2: 0.8335 - jump_sigma_mean_absolute_percentage_error: 197.5624 - jump_sigma_r2: 0.0154 - jump_mu_mean_absolute_percentage_error: 170.2377 - jump_mu_r2: 0.8653 - lambda_mean_absolute_percentage_error: 128.8849 - lambda_r2: 0.1229    
[master 68f8418] updated history, predictions and model
 2 files changed, 0 insertions(+), 0 deletions(-)
Branch master set up to track remote branch master from origin.
Epoch 1/1
10/10 [==============================] - 39s - loss: 0.0239 - sigma_loss: 7.5593e-04 - mu_loss: 0.0095 - jump_sigma_loss: 0.0032 - jump_mu_loss: 0.0103 - lambda_loss: 4.3539e-05 - sigma_mean_absolute_percentage_error: 70.0279 - sigma_r2: 0.7723 - mu_mean_absolute_percentage_er

KeyboardInterrupt: 

# Covnet Single Output Prediction - Lambda Smaller

In [46]:
covnet_lambda = load_model("./Models/covnet_lambda.h5", custom_objects={"r2":r2})

In [47]:
covnet_lambda.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 40, 50, 32)        4640      
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 40, 50, 32)        147488    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 20, 25, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 20, 25, 64)        73792     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 20, 25, 64)        147520    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 10, 12, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 7680)              0         
__________

In [23]:
def lambda_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params()
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append(mp)
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_data = np.reshape(np.array(RETURNS_), (paramset_size*paths_p_paramset, 40, 50, 1))
        
        lambdas = np.array([[mp.lamda] for mp in PARAMS_])
        
        yield train_data, lambdas

lambda_train_gen = lambda_batch_generator(paramset_size=1750, paths_p_paramset=1)

In [ ]:
test_set_pred = standard_convolutional_test_set()

In [ ]:
covnet_lambda.load_weights(H5_DIR_COVNET_LAMBDA)

In [48]:
histories_lambda = np.load("Lambda_histories.npy").tolist()
preds_lambda = np.load("preds_lambda.npy").tolist()

In [49]:
for i in range(15000):
    history = covnet_lambda.fit_generator(lambda_train_gen, steps_per_epoch = 3, epochs = 3)
    
    histories_lambda.append(history.history)
    preds_lambda.append(covnet_lambda.predict(test_set_pred))
    
    covnet_lambda.save("./Models/covnet_lambda.h5")
    if i%10 == 0:
        np.save("Lambda_histories.npy", histories_lambda)
        np.save("preds_lambda.npy", preds_lambda)
        !git add --all
        !git commit -am "Model trained"
        !git push --quiet

Epoch 1/3
3/3 [==============================] - 52s - loss: 7.4174e-06 - mean_absolute_percentage_error: 32.1472 - r2: 0.8554     
Epoch 2/3
3/3 [==============================] - 51s - loss: 7.4208e-06 - mean_absolute_percentage_error: 32.2215 - r2: 0.8567     
Epoch 3/3
3/3 [==============================] - 51s - loss: 7.4522e-06 - mean_absolute_percentage_error: 32.1867 - r2: 0.8556     
[master 2126606] Model trained
 4 files changed, 38 insertions(+), 143 deletions(-)
Git 2.0 from 'matching' to 'simple'. To squelch this message
and maintain the traditional behavior, use:

  git config --global push.default matching

To squelch this message and adopt the new behavior now, use:

  git config --global push.default simple

When push.default is set to 'matching', git will push local branches
to the remote branches that already exist with the same name.

Since Git 2.0, Git defaults to the more conservative 'simple'
behavior, which only pushes the current branch to the corresponding
re

KeyboardInterrupt: 

# Convnet Single Output Prediction - Mu

In [26]:
covnet_mu = load_model("./Models/covnet_mu.h5", custom_objects={"r2":r2})

In [28]:
def jump_mu_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params()
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append(mp)
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_data = np.reshape(np.array(RETURNS_), (paramset_size*paths_p_paramset, 40, 50, 1))
        
        mus = np.array([[mp.gbm_mu] for mp in PARAMS_])
        
        yield train_data, mus

mu_train_gen = jump_mu_batch_generator(paramset_size=1750, paths_p_paramset=1)

In [ ]:
test_set_pred = standard_convolutional_test_set()

In [29]:
histories_covnet_mu = np.load("histories_covnet_mu.npy").tolist()
predictions_covnet_mu = np.load("predictions_covnet_mu.npy").tolist()

In [ ]:
covnet_mu.save("./Models/covnet_mu.h5")

In [30]:
for i in range(150000000):
    history = covnet_mu.fit_generator(mu_train_gen, steps_per_epoch = 10, epochs = 1)
    histories_covnet_mu.append(history.history)
    predictions_covnet_mu.append(covnet_mu.predict(test_set_pred))
    covnet_mu.save("covnet_mu_FULLMODEL.h5")
    
    if (i%10 == 0):
        np.save("histories_covnet_mu.npy", histories_covnet_mu)
        np.save("predictions_covnet_mu.npy", predictions_covnet_mu)

        !git add --all
        !git commit -am "Model trained"
        !git push --quiet

Epoch 1/1
10/10 [==============================] - 171s - loss: 0.0046 - mean_absolute_percentage_error: 52.0238 - r2: 0.9862    
[master 725c1fd] Model trained
 5 files changed, 69 insertions(+), 24 deletions(-)
 rewrite Source_Code/histories_covnet_mu.npy (85%)
 rewrite Source_Code/predictions_covnet_mu.npy (91%)
Git 2.0 from 'matching' to 'simple'. To squelch this message
and maintain the traditional behavior, use:

  git config --global push.default matching

To squelch this message and adopt the new behavior now, use:

  git config --global push.default simple

When push.default is set to 'matching', git will push local branches
to the remote branches that already exist with the same name.

Since Git 2.0, Git defaults to the more conservative 'simple'
behavior, which only pushes the current branch to the corresponding
remote branch that 'git pull' uses to update the current branch.

See 'git help config' and search for 'push.default' for further information.
(the 'simple' mode was

KeyboardInterrupt: 

# Convnet Single Output Prediction - Sigma

In [31]:
covnet_sigma = load_model("./Models/covnet_sigma.h5", custom_objects={"r2":r2})#covnet_single_ReLUs_6_layers()

In [32]:
def sigma_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params()
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append(mp)
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_data = np.reshape(np.array(RETURNS_), (paramset_size*paths_p_paramset, 40, 50, 1))
        
        sigmas = np.array([[mp.all_sigma] for mp in PARAMS_])
        
        yield train_data, sigmas

sigma_train_gen = sigma_batch_generator(paramset_size=1500, paths_p_paramset=1)

In [ ]:
test_set_pred = standard_convolutional_test_set()

In [ ]:
covnet_jump_mu.load_weights(H5_DIR_COVNET_JUMP_MU)

In [33]:
histories_covnet_sigma = np.load("histories_covnet_sigma.npy").tolist()
predictions_covnet_sigma = np.load("predictions_covnet_sigma.npy").tolist()

In [ ]:
#covnet_sigma.save("./Models/covnet_sigma.h5")

In [34]:
for i in range(150000000):
    history = covnet_sigma.fit_generator(sigma_train_gen, steps_per_epoch = 10, epochs = 1)
    histories_covnet_sigma.append(history.history)
    predictions_covnet_sigma.append(covnet_sigma.predict(test_set_pred))
    covnet_sigma.save("covnet_sigma_FULLMODEL.h5")
    
    if (i%10 == 0):
        np.save("histories_covnet_sigma.npy", histories_covnet_sigma)
        np.save("predictions_covnet_sigma.npy", predictions_covnet_sigma)

        !git add --all
        !git commit -am "Model trained"
        !git push --quiet

Epoch 1/1
10/10 [==============================] - 146s - loss: 6.2224e-05 - mean_absolute_percentage_error: 16.7434 - r2: 0.9810    
[master 487b5f3] Model trained
 5 files changed, 74 insertions(+), 14 deletions(-)
 rewrite Source_Code/histories_covnet_sigma.npy (88%)
 rewrite Source_Code/predictions_covnet_sigma.npy (92%)
Git 2.0 from 'matching' to 'simple'. To squelch this message
and maintain the traditional behavior, use:

  git config --global push.default matching

To squelch this message and adopt the new behavior now, use:

  git config --global push.default simple

When push.default is set to 'matching', git will push local branches
to the remote branches that already exist with the same name.

Since Git 2.0, Git defaults to the more conservative 'simple'
behavior, which only pushes the current branch to the corresponding
remote branch that 'git pull' uses to update the current branch.

See 'git help config' and search for 'push.default' for further information.
(the 'simple

KeyboardInterrupt: 

# Convnet Single Output Prediction - Jump Mu

In [35]:
covnet_jump_mu = load_model("./Models/covnet_jump_mu.h5", custom_objects={"r2":r2})#covnet_single_ReLUs_6_layers()

In [36]:
def jump_mu_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params()
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append(mp)
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_data = np.reshape(np.array(RETURNS_), (paramset_size*paths_p_paramset, 40, 50, 1))
        
        jumps_mus = np.array([[mp.jumps_mu] for mp in PARAMS_])
        
        yield train_data, jumps_mus

jump_mu_train_gen = jump_mu_batch_generator(paramset_size=1750, paths_p_paramset=1)

In [ ]:
test_set_pred = standard_convolutional_test_set()

In [ ]:
covnet_jump_mu = load_model(H5_DIR_COVNET_JUMP_MU+"_FULLMODEL.h5", custom_objects={"r2":r2})

In [37]:
histories_covnet_jump_mu = np.load("histories_covnet_jump_mu.npy").tolist()
predictions_covnet_jump_mu = np.load("predictions_covnet_jump_mu.npy").tolist()

In [ ]:
covnet_jump_mu.save("./Models/covnet_jump_mu.h5")

In [38]:
for i in range(150000000):
    history = covnet_jump_mu.fit_generator(jump_mu_train_gen, steps_per_epoch = 10, epochs = 1)
    histories_covnet_jump_mu.append(history.history)
    predictions_covnet_jump_mu.append(covnet_jump_mu.predict(test_set_pred))
    covnet_jump_mu.save("./Models/covnet_jump_mu.h5")
    
    if (i%10 == 0):
        np.save("histories_covnet_jump_mu.npy", histories_covnet_jump_mu)
        np.save("predictions_covnet_jump_mu.npy", predictions_covnet_jump_mu)

        !git add --all
        !git commit -am "Model trained"
        !git push --quiet

Epoch 1/1
10/10 [==============================] - 172s - loss: 0.0031 - mean_absolute_percentage_error: 56.5280 - r2: 0.9636    
[master 0ebbe03] Model trained
 5 files changed, 78 insertions(+), 13 deletions(-)
 rewrite Source_Code/histories_covnet_jump_mu.npy (85%)
Git 2.0 from 'matching' to 'simple'. To squelch this message
and maintain the traditional behavior, use:

  git config --global push.default matching

To squelch this message and adopt the new behavior now, use:

  git config --global push.default simple

When push.default is set to 'matching', git will push local branches
to the remote branches that already exist with the same name.

Since Git 2.0, Git defaults to the more conservative 'simple'
behavior, which only pushes the current branch to the corresponding
remote branch that 'git pull' uses to update the current branch.

See 'git help config' and search for 'push.default' for further information.
(the 'simple' mode was introduced in Git 1.7.11. Use the similar mode


KeyboardInterrupt: 

# Convnet Single Output Prediction - Jump Sigma

In [9]:
def r2(y_true, y_pred):
    """
    returns the correlation coefficient of y_pred against y_true.

    :param y_true: the true values (independent variable)
    :param y_pred: the predicted values (dependent variable)
    """
    
    SSE = K.sum(K.square(y_true-y_pred))
    SST = K.sum(K.square(y_true-K.mean(y_true)))
    
    return 1-SSE/SST

covnet_jump_sigma.compile('adam', 'mean_squared_error', metrics=['mean_absolute_percentage_error', r2])

NameError: name 'covnet_jump_sigma' is not defined

In [43]:
covnet_jump_sigma.compile('adam', 'mean_squared_error', metrics=['mean_absolute_percentage_error', r2])

In [ ]:
keras.__version__

In [39]:
covnet_jump_sigma = load_model("./Models/covnet_jump_sigma.h5", custom_objects={"r2":r2})#covnet_single_ReLUs_6_layers()
#covnet_jump_sigma.load_weights(H5_DIR_COVNET_JUMP_SIGMA)

In [40]:
def jump_sigma_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params()
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append(mp)
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_data = np.reshape(np.array(RETURNS_), (paramset_size*paths_p_paramset, 40, 50, 1))
        
        jumps_sigmas = np.array([[mp.jumps_sigma] for mp in PARAMS_])
        
        yield train_data, jumps_sigmas

jump_sigma_train_gen = jump_sigma_batch_generator(paramset_size=1750, paths_p_paramset=1)

In [ ]:
test_set_pred = standard_convolutional_test_set()

In [41]:
histories_covnet_jump_sigma = np.load("histories_covnet_jump_sigma.npy").tolist()
predictions_covnet_jump_sigma = np.load("predictions_covnet_jump_sigma.npy").tolist()
    
#histories_covnet_jump_sigma = []
#predictions_covnet_jump_sigma = []

In [ ]:
covnet_jump_sigma.save("./Models/covnet_jump_sigma.h5")

In [ ]:
for i in range(100000):
    
    history = covnet_jump_sigma.fit_generator(jump_sigma_train_gen, steps_per_epoch = 3, epochs = 3)
    histories_covnet_jump_sigma.append(history.history)
    predictions_covnet_jump_sigma.append(covnet_jump_sigma.predict(test_set_pred))
    covnet_jump_sigma.save("./Models/covnet_jump_sigma.h5")
    
    np.save("histories_covnet_jump_sigma.npy", histories_covnet_jump_sigma)
    np.save("predictions_covnet_jump_sigma.npy", predictions_covnet_jump_sigma)
    
    if i % 5 == 0:
        !cd ..
        !git config --global user.email "rwmmostert@gmail.com"
        !git config --global user.name "RWMostert"
        !git add --all
        !git commit -am "updated history, predictions and model"
        !git push -u origin master --quiet

Epoch 1/3
3/3 [==============================] - 52s - loss: 7.3159e-04 - mean_absolute_percentage_error: 53.4050 - r2: 0.7824     
Epoch 2/3
3/3 [==============================] - 51s - loss: 6.6728e-04 - mean_absolute_percentage_error: 55.9392 - r2: 0.8002     
Epoch 3/3


# Ensembling Network

In [ ]:
ff_ensembling = Sequential()
ff_ensembling.add(Dense(128, input_dim = 8, activation='relu'))
ff_ensembling.add(Dense(256, activation='relu'))
ff_ensembling.add(Dense(128, activation='relu'))
ff_ensembling.add(Dense(64, activation='relu'))
ff_ensembling.add(Dense(32, activation='relu'))
ff_ensembling.add(Dense(5))

ff_ensembling.compile('adam', 'mean_squared_error', metrics=['mean_absolute_percentage_error', r2])

In [ ]:
all_time = 2000
paramset_size = 1
paths_p_paramset = 1

RETURNS_ = []
PARAMS_ = []
        
for i in range(paramset_size):
    mp = random_model_params()
    mp.all_time = all_time
            
    for j in range(paths_p_paramset):
        PARAMS_.append([mp.all_sigma, mp.gbm_mu, mp.jumps_sigma, mp.jumps_mu, mp.lamda])
        RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
train_data = np.reshape(np.array(RETURNS_), (paramset_size*paths_p_paramset, 40, 50, 1))
                
predictions = []
for return_set in train_data:
    output1 = covnet_mo.predict(np.array([return_set]))
    output2 = covnet_lambda.predict(np.array([return_set]))
    output3 = covnet_jump_mu.predict(np.array([return_set]))
    output4 = covnet_jump_sigma.predict(np.array([return_set]))
    for output in output1:
        predictions.append(output[0][0])
    predictions.append(output2[0][0])
    predictions.append(output3[0][0])
    predictions.append(output4[0][0])
        
print(predictions)
        
print(PARAMS_)

In [ ]:
def ensemble_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params()
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append([mp.all_sigma, mp.gbm_mu, mp.jumps_sigma, mp.jumps_mu, mp.lamda])
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_data = np.reshape(np.array(RETURNS_), (paramset_size*paths_p_paramset, 40, 50, 1))
                
        predictionset = []
        for return_set in train_data:
            prediction = []
            output1 = covnet_mo.predict(np.array([return_set]))
            output2 = covnet_lambda.predict(np.array([return_set]))
            output3 = covnet_jump_mu.predict(np.array([return_set]))
            output4 = covnet_jump_sigma.predict(np.array([return_set]))
            for output in output1:
                prediction.append(output[0][0])
            prediction.append(output2[0][0])
            prediction.append(output3[0][0])
            prediction.append(output4[0][0])
            predictionset.append(prediction)
        
        
        yield np.array(predictionset), np.array(PARAMS_)

ensemble_batch_gen = ensemble_batch_generator(paramset_size=10, paths_p_paramset=5)

In [ ]:
ff_ensembling.load_weights(H5_DIR_ENSEMBLING)

In [ ]:


for i in range(150):
    ff_ensembling.fit_generator(ensemble_batch_gen, steps_per_epoch = 10, epochs = 1)
    ff_ensembling.save_weights(H5_DIR_ENSEMBLING)

# Sensitivity Tests

### Sensitivity Test: Mu

In [ ]:
mu_s = []
preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []


preds_SO_MEAN = []
preds_SO_Upper_84 = []
preds_SO_Lower_16 = []

In [ ]:
for j in range(100):
    mp = test_set_model_params()
    mp.gbm_mu = -1 + float(j)/50
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set(mp=mp)
    
    
    print("PREDICTION...\n")
    predictions_lambda = covnet_lambda.predict(test_set_pred)
    predictions_jump_mu = covnet_jump_mu.predict(test_set_pred)
    predictions_jump_sigma = covnet_jump_sigma.predict(test_set_pred)
    predictions_sigma = covnet_sigma.predict(test_set_pred)
    predictions_mu = covnet_mu.predict(test_set_pred)
    
    predictions_so = [predictions_sigma, predictions_mu, predictions_jump_sigma, predictions_jump_mu, predictions_lambda]
    
    preds_SO_MEAN.append(np.mean(predictions_so, axis = 1))
    preds_SO_Upper_84.append(np.percentile(predictions_so, 84, axis = 1))
    preds_SO_Lower_16.append(np.percentile(predictions_so, 16, axis = 1))
    
    mu_s.append(mp.gbm_mu)
    
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
    print(np.transpose(np.mean(predictions_so, axis = 1)))
    print("   sigma,       mu,      jump_sigma,   jump_mu,    lambda")
    print("    0.1         "+str(mp.gbm_mu)+"     0.07              0.05       0.02")
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Mu_preds_Upper_84.npy", preds_SO_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Mu_preds_Lower_16.npy", preds_SO_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Mu_preds_MEAN.npy", preds_SO_MEAN)
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Mu_preds_Upper_84.npy", preds_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Mu_preds_Lower_16.npy", preds_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Mu_preds_MEAN.npy", preds_MEAN)
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_lambda_Mu_mu_s.npy", mu_s)

In [ ]:
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Mu_preds_Upper_84.npy", preds_SO_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Mu_preds_Lower_16.npy", preds_SO_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Mu_preds_MEAN.npy", preds_SO_MEAN)
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Mu_preds_Upper_84.npy", preds_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Mu_preds_Lower_16.npy", preds_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Mu_preds_MEAN.npy", preds_MEAN)
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_lambda_Mu_mu_s.npy", mu_s)

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigma_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigma_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

sigmas_single = np.array(preds_SO_MEAN)[:,0].reshape(100)
sigma_upper_84_single = np.array(preds_SO_Upper_84)[:,0].reshape(100)
sigma_lower_16_single = np.array(preds_SO_Lower_16)[:,0].reshape(100)

In [ ]:
fig = pylab.figure()
sns.set_style("white")
pylab.plot(mu_s, sigmas, color='r', label="predicted value");
pylab.axhline(y=0.1, color='black', linestyle='-', label="actual value")
pylab.fill_between(mu_s, sigma_lower_16, sigma_upper_84, color='r', alpha=0.1, edgecolor='none')
pylab.xlabel("Mu")
pylab.ylabel("Sigma")

pylab.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Mu/"+"sigma.png")

In [ ]:
fig = pylab.figure()
sns.set_style("white")
pylab.plot(mu_s, sigmas_single, color='r', label="predicted value");
pylab.axhline(y=0.1, color='black', linestyle='-', label="actual value")
pylab.fill_between(mu_s, sigma_lower_16_single, sigma_upper_84_single, color='r', alpha=0.1, edgecolor='none')
pylab.xlabel("Mu")
pylab.ylabel("Sigma")

pylab.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-SingleOutput/Varying-Mu/"+"sigma.png")

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

In [ ]:
fig = pylab.figure()
pylab.plot(mu_s, jump_sigmas, color='b', label="predicted value");
pylab.axhline(y=0.07, color='black', linestyle='-', label="actual value")
pylab.fill_between(mu_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
pylab.xlabel("Mu")
pylab.ylabel("Jumps Sigma")

pylab.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Mu/"+"Jumps_Sigma.png")

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

In [ ]:
fig = pylab.figure()
pylab.plot(mu_s, jump_mus, color='g', label="predicted value");
pylab.axhline(y=0.05, color='black', linestyle='-', label="actual value")
pylab.fill_between(mu_s, jump_mus_lower_16, jump_mus_upper_84, color='g', alpha=0.1, edgecolor='none')
pylab.xlabel("Mu")
pylab.ylabel("Jumps Mu")

pylab.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Mu/"+"Jumps_Mu.png")

In [ ]:
lambdas = np.array(preds_MEAN)[:,4].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,4].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,4].reshape(100)

In [ ]:
fig = pylab.figure()
pylab.plot(mu_s, lambdas, color='y', label="predicted value");
pylab.axhline(y=0.02, color='black', linestyle='-', label="actual value")
pylab.fill_between(mu_s, lambdas_lower_16, lambdas_upper_84, color='y', alpha=0.2, edgecolor='none')
pylab.xlabel("Mu")
pylab.ylabel("Lambda")

pylab.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Mu/"+"Lambda.png")

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)

In [ ]:
fig = pylab.figure()
pylab.plot(mu_s, mus_-mu_s, color='g', label="predicted value");
#pylab.plot(0, mu_s, color='black', alpha = 0.5, label="actual value");
pylab.axhline(y=0, color='black', linestyle='-', label="actual value")
pylab.fill_between(mu_s, mus_lower_16-mu_s, mus_upper_84-mu_s, color='g', alpha=0.1, edgecolor='none')
pylab.xlabel("Mu")
pylab.ylabel("Prediction Deviation from Mu")
ax = pylab.gca()
vals = ax.get_yticks()
ax.set_yticklabels(['-'+str(x) for x in vals[vals<0]] + [0] + ['+'+str(x) for x in vals[vals>0]])

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Mu/"+"Mu.png")

# Sensitivity Test - Sigma

In [ ]:
sigma_s = []

preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []

preds_SO_MEAN = []
preds_SO_Upper_84 = []
preds_SO_Lower_16 = []

for j in range(100):
    mp = test_set_model_params()
    mp.all_sigma = 0.001 + j*(0.2 - 0.001)/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set(mp=mp)
    
    
    print("PREDICTION...\n")
    predictions_lambda = covnet_lambda.predict(test_set_pred)
    predictions_jump_mu = covnet_jump_mu.predict(test_set_pred)
    predictions_jump_sigma = covnet_jump_sigma.predict(test_set_pred)
    predictions_sigma = covnet_sigma.predict(test_set_pred)
    predictions_mu = covnet_mu.predict(test_set_pred)
    
    predictions_so = [predictions_sigma, predictions_mu, predictions_jump_sigma, predictions_jump_mu, predictions_lambda]
    
    preds_SO_MEAN.append(np.mean(predictions_so, axis = 1))
    preds_SO_Upper_84.append(np.percentile(predictions_so, 84, axis = 1))
    preds_SO_Lower_16.append(np.percentile(predictions_so, 16, axis = 1))
    
    sigma_s.append(mp.all_sigma)
    
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
    print(np.transpose(np.mean(predictions_so, axis = 1)))
    print("   sigma,       mu,      jump_sigma,   jump_mu,    lambda")
    print("   "+str(mp.all_sigma)+"      0.05       0.07              0.05       0.02")
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-Covnet_MO_Sigma_sigma_s.npy", sigma_s)
   
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Sigma_preds_Upper_84.npy", preds_SO_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Sigma_preds_Lower_16.npy", preds_SO_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Sigma_preds_MEAN.npy", preds_SO_MEAN)
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Sigma_preds_Upper_84.npy", preds_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Sigma_preds_Lower_16.npy", preds_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Sigma_preds_MEAN.npy", preds_MEAN)

In [ ]:
sigma_s = []
preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []

for j in range(100):
    covnet_mo_elu = covnet_multiple_output_with_ELUs()
    covnet_mo_elu.load_weights(H5_DIR_COVNET_MULTIPLE_ELU)
    
    mp.all_sigma = 0.001 + j*(0.2 - 0.001)/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set()
    
    print("PREDICTION...\n")
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    print("   sigma,    mu,      jump_sigma,      jump_mu,    lambda")
    print("   "+str(mp.all_sigma)+"      0.05       0.07              0.05       0.02")
    print(np.mean(predictions_elu, axis = 1)+np.std(predictions_elu, axis = 1))
    print(np.percentile(predictions_elu, 84, axis = 1))
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    sigma_s.append(mp.all_sigma)
    
np.save("Sensitivity_Test-Covnet_MO_Sigma_sigma_s.npy", sigma_s)
np.save("Sensitivity_Test-Covnet_MO_Sigma_preds_MEAN.npy", preds_MEAN)
np.save("Sensitivity_Test-Covnet_MO_Sigma_preds_Upper_84.npy", preds_Upper_84)
np.save("Sensitivity_Test-Covnet_MO_Sigma_preds_Lower_16.npy", preds_Lower_16)

In [ ]:
sigma_s = np.load("Sensitivity_Test-Covnet_MO_Sigma_sigma_s.npy")
preds_MEAN = np.load("Sensitivity_Test-Covnet_MO_Sigma_preds_MEAN.npy")
preds_Upper_84 = np.load("Sensitivity_Test-Covnet_MO_Sigma_preds_Upper_84.npy")
preds_Lower_16 = np.load("Sensitivity_Test-Covnet_MO_Sigma_preds_Lower_16.npy")

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigma_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigma_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

fig = pylab.figure()
pylab.plot(sigma_s, sigmas-sigma_s, color='r', label="predicted value");
#pylab.plot(sigma_s, sigma_s, color='black', label="actual value");
pylab.axhline(y=0, color='black', linestyle='-', label="actual value")
pylab.fill_between(sigma_s, sigma_lower_16-sigma_s, sigma_upper_84-sigma_s, color='r', alpha=0.1, edgecolor='none')
pylab.xlabel("Sigma")
pylab.ylabel("Prediction Deviation from Sigma")
ax = pylab.gca()
vals = ax.get_yticks()
ax.set_yticklabels(['-'+str(x) for x in vals[vals<0]] + [0] + ['+'+str(x) for x in vals[vals>0]])


pylab.legend(bbox_to_anchor=(1, 0.185), loc=0, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Sigma/"+"Sigma.png")

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

fig = pylab.figure()
pylab.plot(sigma_s, jump_sigmas, color='b', label="predicted value");
pylab.axhline(y=0.07, color='black', linestyle='-', label="actual value")
pylab.fill_between(sigma_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
pylab.xlabel("Sigma")
pylab.ylabel("Jumps Sigma")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Sigma/"+"Jumps_Sigma.png")

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

fig = pylab.figure()
pylab.plot(sigma_s, jump_mus, color='g', label="predicted value");
pylab.axhline(y=0.05, color='black', linestyle='-', label="actual value")
pylab.fill_between(sigma_s, jump_mus_lower_16, jump_mus_upper_84, color='g', alpha=0.1, edgecolor='none')
pylab.xlabel("Sigma")
pylab.ylabel("Jumps Mu")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Sigma/"+"Jumps_Mu.png")

In [ ]:
lambdas = np.array(preds_MEAN)[:,4].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,4].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,4].reshape(100)

fig = pylab.figure()
pylab.plot(sigma_s, lambdas, color='y', label="predicted value");
pylab.axhline(y=0.02, color='black', linestyle='-', label="actual value")
pylab.fill_between(sigma_s, lambdas_lower_16, lambdas_upper_84, color='y', alpha=0.2, edgecolor='none')
pylab.xlabel("Sigma")
pylab.ylabel("Lambda")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Sigma/"+"Lambda.png")

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)

fig = pylab.figure()
pylab.plot(sigma_s, mus_, color='g', label="predicted value");
pylab.axhline(y=0.05, color='black', linestyle='-', label="actual value")
pylab.fill_between(sigma_s, mus_lower_16, mus_upper_84, color='g', alpha=0.1, edgecolor='none')
pylab.xlabel("Sigma")
pylab.ylabel("Mu")

pylab.legend(bbox_to_anchor=(0.4085, 1), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Sigma/"+"Mu.png")

# Sensitivity Test - Jumps Sigma

In [ ]:
jump_sigma_s = []

preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []

preds_SO_MEAN = []
preds_SO_Upper_84 = []
preds_SO_Lower_16 = []

for j in range(100):
    mp = test_set_model_params()
    mp.jumps_sigma = 0.001 + j*(0.2 - 0.001)/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set(mp=mp)
    
    
    print("PREDICTION...\n")
    predictions_lambda = covnet_lambda.predict(test_set_pred)
    predictions_jump_mu = covnet_jump_mu.predict(test_set_pred)
    predictions_jump_sigma = covnet_jump_sigma.predict(test_set_pred)
    predictions_sigma = covnet_sigma.predict(test_set_pred)
    predictions_mu = covnet_mu.predict(test_set_pred)
    
    predictions_so = [predictions_sigma, predictions_mu, predictions_jump_sigma, predictions_jump_mu, predictions_lambda]
    
    preds_SO_MEAN.append(np.mean(predictions_so, axis = 1))
    preds_SO_Upper_84.append(np.percentile(predictions_so, 84, axis = 1))
    preds_SO_Lower_16.append(np.percentile(predictions_so, 16, axis = 1))
    
    jump_sigma_s.append(mp.jumps_sigma)
    
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
    print(np.transpose(np.mean(predictions_so, axis = 1)))
    print("   sigma,       mu,      jump_sigma,   jump_mu,    lambda")
    print("   0.1      0.05       "+str(mp.jumps_sigma)+"              0.05       0.02")
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    
np.save("Sensitivity_Test-Covnet_MO_JumpSigma_jumpsigma_s.npy", jump_sigma_s)
   
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-JumpSigma_preds_Upper_84.npy", preds_SO_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-JumpSigma_preds_Lower_16.npy", preds_SO_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-JumpSigma_preds_MEAN.npy", preds_SO_MEAN)
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-JumpSigma_preds_Upper_84.npy", preds_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-JumpSigma_preds_Lower_16.npy", preds_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-JumpSigma_preds_MEAN.npy", preds_MEAN)

In [ ]:
jump_sigma_s = []
preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []

for j in range(100):
    covnet_mo_elu = covnet_multiple_output_with_ELUs()
    covnet_mo_elu.load_weights(H5_DIR_COVNET_MULTIPLE_ELU)
    
    mp.jumps_sigma = 0.001 + j*(0.2 - 0.001)/100
    test_set = []
    
    print(str(j) + " out of "+str(100))
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set()
    
    
    print("PREDICTION...\n")
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    print("   sigma,    mu,      jump_sigma,      jump_mu,    lambda")
    print("   0.1      0.05       "+str(mp.jumps_sigma)+"              0.05       0.02")
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    jump_sigma_s.append(mp.jumps_sigma)
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
np.save("Sensitivity_Test-Covnet_MO_JumpSigma_preds_Upper_84.npy", preds_Upper_84)
np.save("Sensitivity_Test-Covnet_MO_JumpSigma_preds_Lower_16.npy", preds_Lower_16)
np.save("Sensitivity_Test-Covnet_MO_JumpSigma_jump_sigma_s.npy", jump_sigma_s)
np.save("Sensitivity_Test-Covnet_MO_JumpSigma_preds_MEAN.npy", preds_MEAN)

In [ ]:
preds_Upper_84 = np.load("Sensitivity_Test-Covnet_MO_JumpSigma_preds_Upper_84.npy")
preds_Lower_16 = np.load("Sensitivity_Test-Covnet_MO_JumpSigma_preds_Lower_16.npy")
jump_sigma_s = np.load("Sensitivity_Test-Covnet_MO_JumpSigma_jump_sigma_s.npy")
preds_MEAN = np.load("Sensitivity_Test-Covnet_MO_JumpSigma_preds_MEAN.npy")

In [ ]:
def sensitivity_plot_w_true_value(x_values, predictions, true_value, upper_confidence_predictions, lower_confidence_predictions, 
                                  x_name, y_name, line_color = 'b', save_dir = None, legend_anchor = (1,1)):
    fig = pylab.figure()
    pylab.plot(x_values, predictions, color=line_color, label="predicted value");
    pylab.axhline(y=true_value, color='black', linestyle='-', label="actual value")
    pylab.fill_between(x_values, mus_lower_16, mus_upper_84, color=line_color, alpha=0.1, edgecolor='none')
    pylab.xlabel(x_name)
    pylab.ylabel(y_name)

    pylab.legend(bbox_to_anchor=legend_anchor, loc=1, borderaxespad=0.)
    if save_dir != None:
        fig.savefig(save_dir)
    
    return fig

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)
save_dir = ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_sigma/"+"Mu.png"

sensitivity_plot_w_true_value(jump_sigma_s, mus_, test_set_model_params().jumps_sigma, mus_upper_84, mus_lower_16, 
                              "Jumps Mu", "Mu", line_color = 'g', save_dir = "./Mu.png")

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigmas_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigmas_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

fig = pylab.figure()
pylab.plot(jump_sigma_s, sigmas, color='r', label="predicted value");
pylab.axhline(y=0.1, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_sigma_s, sigmas_lower_16, sigmas_upper_84, color='r', alpha=0.1, edgecolor='none')
pylab.xlabel("Jumps Sigma")
pylab.ylabel("Sigma")

pylab.legend(bbox_to_anchor=(1, 1), loc=0, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_sigma/"+"Sigma.png")

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

fig = pylab.figure()
pylab.plot(jump_sigma_s, jump_sigmas, color='b', label="predicted value");
pylab.plot(jump_sigma_s, jump_sigma_s, color='black', label="actual value");
pylab.fill_between(jump_sigma_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
pylab.xlabel("Jumps Sigma")
pylab.ylabel("Jumps Sigma")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_sigma/"+"Jumps_Sigma.png")

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

fig = pylab.figure()
pylab.plot(jump_sigma_s, jump_mus, color='g', label="predicted value");
pylab.axhline(y=0.05, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_sigma_s, jump_mus_lower_16, jump_mus_upper_84, color='g', alpha=0.1, edgecolor='none')
pylab.xlabel("Jumps Sigma")
pylab.ylabel("Jumps Mu")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_sigma/"+"Jumps_Mu.png")

In [ ]:
lambdas = np.array(preds_MEAN)[:,4].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,4].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,4].reshape(100)

fig = pylab.figure()
pylab.plot(jump_sigma_s, lambdas, color='y', label="predicted value");
pylab.axhline(y=0.02, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_sigma_s, lambdas_lower_16, lambdas_upper_84, color='y', alpha=0.2, edgecolor='none')
pylab.xlabel("Jumps Sigma")
pylab.ylabel("Lambda")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_sigma/"+"Lambda.png")

# Sensitivity Test - Jumps Mu

In [ ]:
jump_mu_s = []

preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []

preds_SO_MEAN = []
preds_SO_Upper_84 = []
preds_SO_Lower_16 = []

for j in range(100):
    mp = test_set_model_params()
    mp.jumps_mu = -0.5 + j/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set(mp=mp)
    
    
    print("PREDICTION...\n")
    predictions_lambda = covnet_lambda.predict(test_set_pred)
    predictions_jump_mu = covnet_jump_mu.predict(test_set_pred)
    predictions_jump_sigma = covnet_jump_sigma.predict(test_set_pred)
    predictions_sigma = covnet_sigma.predict(test_set_pred)
    predictions_mu = covnet_mu.predict(test_set_pred)
    
    predictions_so = [predictions_sigma, predictions_mu, predictions_jump_sigma, predictions_jump_mu, predictions_lambda]
    
    preds_SO_MEAN.append(np.mean(predictions_so, axis = 1))
    preds_SO_Upper_84.append(np.percentile(predictions_so, 84, axis = 1))
    preds_SO_Lower_16.append(np.percentile(predictions_so, 16, axis = 1))
    
    jump_mu_s.append(mp.jumps_mu)
    
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
    print(np.transpose(np.mean(predictions_so, axis = 1)))
    print("   sigma,       mu,      jump_sigma,   jump_mu,    lambda")
    print("   0.1      0.05       0.07              "+str(mp.jumps_mu)+"       0.02")
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    
np.save("Sensitivity_Test-Covnet_MO_JumpMu_jump_mu_s.npy", jump_mu_s)

np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-JumpMu_preds_Upper_84.npy", preds_SO_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-JumpMu_preds_Lower_16.npy", preds_SO_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-JumpMu_preds_MEAN.npy", preds_SO_MEAN)
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-JumpMu_preds_Upper_84.npy", preds_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-JumpMu_preds_Lower_16.npy", preds_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-JumpMu_preds_MEAN.npy", preds_MEAN)

In [ ]:
jump_mu_s = []
preds_MEAN = []
preds_Upper_84 = []
preds_Lower_16 = []

for j in range(100):
    covnet_mo_elu = covnet_multiple_output_with_ELUs()
    covnet_mo_elu.load_weights(H5_DIR_COVNET_MULTIPLE_ELU)
    
    mp.jumps_mu = -0.5 + j/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set()
    
    
    print("PREDICTION...\n")
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    print("   sigma,    mu,      jump_sigma,      jump_mu,    lambda")
    print("   0.1      0.05       0.07              "+str(mp.jumps_mu)+"       0.02")
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    jump_mu_s.append(mp.jumps_mu)
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
np.save("Sensitivity_Test-Covnet_MO_JumpMu_preds_Upper_84.npy", preds_Upper_84)
np.save("Sensitivity_Test-Covnet_MO_JumpMu_preds_Lower_16.npy", preds_Lower_16)
    
np.save("Sensitivity_Test-Covnet_MO_JumpMu_jump_mu_s.npy", jump_mu_s)
np.save("Sensitivity_Test-Covnet_MO_JumpMu_preds_MEAN.npy", preds_MEAN)

In [ ]:
preds_Upper_84 = np.load("Sensitivity_Test-Covnet_MO_JumpMu_preds_Upper_84.npy")
preds_Lower_16 = np.load("Sensitivity_Test-Covnet_MO_JumpMu_preds_Lower_16.npy")
    
jump_mu_s = np.load("Sensitivity_Test-Covnet_MO_JumpMu_jump_mu_s.npy")
preds_MEAN = np.load("Sensitivity_Test-Covnet_MO_JumpMu_preds_MEAN.npy")

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)

fig = pylab.figure()
pylab.plot(jump_mu_s, mus_, color='g', label="predicted value");
pylab.axhline(y=0.05, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_mu_s, mus_lower_16, mus_upper_84, color='g', alpha=0.1, edgecolor='none')
pylab.xlabel("Jumps Mu")
pylab.ylabel("Mu")
pylab.xlim(-0.5, 0.5)

pylab.legend(bbox_to_anchor=(1, 0.19), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_mu/"+"Mu.png")

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigmas_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigmas_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

fig = pylab.figure()
pylab.plot(jump_mu_s, sigmas, color='r', label="predicted value");
pylab.axhline(y=0.1, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_mu_s, sigmas_lower_16, sigmas_upper_84, color='r', alpha=0.1, edgecolor='none')
pylab.xlabel("Jumps Mu")
pylab.ylabel("Sigma")
pylab.xlim(-0.5, 0.5)

pylab.legend(bbox_to_anchor=(1, 0.185), loc=0, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_mu/"+"Sigma.png")

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

fig = pylab.figure()
pylab.plot(jump_mu_s, jump_sigmas, color='b', label="predicted value");
pylab.axhline(y=0.07, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_mu_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
pylab.xlabel("Jumps Mu")
pylab.ylabel("Jumps Sigma")
pylab.xlim(-0.5, 0.5)

pylab.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_mu/"+"Jumps_Sigma.png")

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

fig = pylab.figure()
pylab.plot(jump_mu_s, jump_mus-jump_mu_s, color='g', label="predicted value");
pylab.axhline(y=0, color='black', linestyle='-', label="actual value")

#pylab.plot(jump_mu_s, jump_mu_s, color='black', label="actual value");
pylab.fill_between(jump_mu_s, jump_mus_lower_16-jump_mu_s, jump_mus_upper_84-jump_mu_s, color='g', alpha=0.1, edgecolor='none')
pylab.xlabel("Jumps Mu")
pylab.ylabel("Prediction Deviation from Jumps Mu")
pylab.xlim(-0.5, 0.5)
ax = pylab.gca()
vals = ax.get_yticks()
ax.set_yticklabels(['-'+str(x) for x in vals[vals<0]] + [0] + ['+'+str(x) for x in vals[vals>0]])


pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_mu/"+"Jumps_Mu.png")

In [ ]:
lambdas = np.array(preds_MEAN)[:,4].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,4].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,4].reshape(100)

fig = pylab.figure()
pylab.plot(jump_mu_s, lambdas, color='y', label="predicted value");
pylab.axhline(y=0.02, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_mu_s, lambdas_lower_16, lambdas_upper_84, color='y', alpha=0.2, edgecolor='none')
pylab.xlabel("Jumps Mu")
pylab.ylabel("Lambda")
pylab.xlim(-0.5, 0.5)

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Jumps_mu/"+"Lambda.png")

# Sensitivity Test - Lambda

In [ ]:
jump_lambda_s = []

preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []

preds_SO_MEAN = []
preds_SO_Upper_84 = []
preds_SO_Lower_16 = []

for j in range(100):
    mp = test_set_model_params()
    mp.lamda = 0.0001 + j*(0.025 - 0.0001)/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set(mp=mp)
    
    
    print("PREDICTION...\n")
    predictions_lambda = covnet_lambda.predict(test_set_pred)
    predictions_jump_mu = covnet_jump_mu.predict(test_set_pred)
    predictions_jump_sigma = covnet_jump_sigma.predict(test_set_pred)
    predictions_sigma = covnet_sigma.predict(test_set_pred)
    predictions_mu = covnet_mu.predict(test_set_pred)
    
    predictions_so = [predictions_sigma, predictions_mu, predictions_jump_sigma, predictions_jump_mu, predictions_lambda]
    
    preds_SO_MEAN.append(np.mean(predictions_so, axis = 1))
    preds_SO_Upper_84.append(np.percentile(predictions_so, 84, axis = 1))
    preds_SO_Lower_16.append(np.percentile(predictions_so, 16, axis = 1))
    
    jump_mu_s.append(mp.jumps_mu)
    
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
    print(np.transpose(np.mean(predictions_so, axis = 1)))
    print("   sigma,       mu,      jump_sigma,   jump_mu,    lambda")
    print("   0.1      0.05       0.07              0.05       "+str(mp.lamda))
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    
np.save("Sensitivity_Test-Covnet_MO_Lambda_jump_lambda_s.npy", jump_lambda_s)


np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Lambda_preds_Upper_84.npy", preds_SO_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Lambda_preds_Lower_16.npy", preds_SO_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_SO-Lambda_preds_MEAN.npy", preds_SO_MEAN)
    
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Lambda_preds_Upper_84.npy", preds_Upper_84)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Lambda_preds_Lower_16.npy", preds_Lower_16)
np.save("./Sensitivity-Test-Results/Sensitivity_Test-covnet_MultipleO-Lambda_preds_MEAN.npy", preds_MEAN)

In [ ]:
jump_lambda_s = []
preds_MEAN = []
preds_Upper_84 = []
preds_Lower_16 = []

for j in range(100):
    covnet_mo_elu = covnet_multiple_output_with_ELUs()
    covnet_mo_elu.load_weights(H5_DIR_COVNET_MULTIPLE_ELU)
    
    mp.lamda = 0.0001 + j*(0.025 - 0.0001)/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set()
    
    
    print("PREDICTION...\n")
    predictions_elu = covnet_mo_elu.predict(test_set_pred)
    
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    print("   sigma,    mu,      jump_sigma,      jump_mu,    lambda")
    print("   0.1      0.05       0.07              0.05       "+str(mp.lamda))
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    jump_lambda_s.append(mp.lamda)
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
np.save("Sensitivity_Test-Covnet_MO_Lambda_preds_Upper_84.npy", preds_Upper_84)
np.save("Sensitivity_Test-Covnet_MO_Lambda_preds_Lower_16.npy", preds_Lower_16)
    
np.save("Sensitivity_Test-Covnet_MO_Lambda_jump_lambda_s.npy", jump_lambda_s)
np.save("Sensitivity_Test-Covnet_MO_Lambda_preds_MEAN.npy", preds_MEAN)

In [ ]:
preds_Upper_84 = np.load("Sensitivity_Test-Covnet_MO_Lambda_preds_Upper_84.npy")
preds_Lower_16 = np.load("Sensitivity_Test-Covnet_MO_Lambda_preds_Lower_16.npy")
    
jump_lambda_s = np.load("Sensitivity_Test-Covnet_MO_Lambda_jump_lambda_s.npy")
preds_MEAN = np.load("Sensitivity_Test-Covnet_MO_Lambda_preds_MEAN.npy")

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)

fig = pylab.figure()
pylab.plot(jump_lambda_s, mus_, color='g', label="predicted value");
pylab.axhline(y=0.05, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_lambda_s, mus_lower_16, mus_upper_84, color='g', alpha=0.1, edgecolor='none')
pylab.xlabel("Lambda")
pylab.ylabel("Mu")

pylab.legend(bbox_to_anchor=(1, 0.19), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Lambda/"+"Mu.png")

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigmas_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigmas_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

fig = pylab.figure()
pylab.plot(jump_lambda_s, sigmas, color='r', label="predicted value");
pylab.axhline(y=0.1, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_lambda_s, sigmas_lower_16, sigmas_upper_84, color='r', alpha=0.1, edgecolor='none')
pylab.xlabel("Lambda")
pylab.ylabel("Sigma")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=0, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Lambda/"+"Sigma.png")

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

fig = pylab.figure()
pylab.plot(jump_lambda_s, jump_sigmas, color='b', label="predicted value");
pylab.axhline(y=0.07, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_lambda_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
pylab.xlabel("Lambda")
pylab.ylabel("Jumps Sigma")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Lambda/"+"Jumps_Sigma.png")

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

fig = pylab.figure()
pylab.plot(jump_lambda_s, jump_mus, color='g', label="predicted value");
pylab.axhline(y=0.05, color='black', linestyle='-', label="actual value")
pylab.fill_between(jump_lambda_s, jump_mus_lower_16, jump_mus_upper_84, color='g', alpha=0.1, edgecolor='none')
pylab.xlabel("Lambda")
pylab.ylabel("Jumps Mu")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Lambda/"+"Jumps_Mu.png")

In [ ]:
lambdas = np.array(preds_MEAN)[:,4].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,4].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,4].reshape(100)

fig = pylab.figure()
pylab.plot(jump_lambda_s, lambdas, color='y', label="predicted value");
pylab.plot(jump_lambda_s, jump_lambda_s, color='black', label="actual value");
pylab.fill_between(jump_lambda_s, lambdas_lower_16, lambdas_upper_84, color='y', alpha=0.2, edgecolor='none')
pylab.xlabel("Lambda")
pylab.ylabel("Lambda")

pylab.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Lambda/"+"Lambda.png")

# Sensitivity Tests

### Sensitivity Test: Mu

In [ ]:
mu_s = []
preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []


preds_SINGLE_LAMBDA_MEAN = []
preds_SINGLE_LAMBDA_Lower_16 = []
preds_SINGLE_LAMBDA_Upper_84 = []

In [ ]:
for j in range(100):
    covnet_lambda = covnet_single_lambda()
    covnet_lambda.load_weights(H5_DIR_COVNET_LAMBDA)
    
    mp.gbm_mu = -1 + float(j)/50
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set()
    
    
    print("PREDICTION...\n")
    predictions_elu = covnet_lambda.predict(test_set_pred)
    
    #predictions_lambda = covnet_lambda.predict(test_set_pred)
    
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    print("   sigma,       mu,      jump_sigma,   jump_mu,    lambda")
    print("    0.1         "+str(mp.gbm_mu)+"     0.07              0.05       0.02")
    
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    mu_s.append(mp.gbm_mu)
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
np.save("Sensitivity_Test-covnet_lambda_Mu_preds_Upper_84.npy", preds_Upper_84)
np.save("Sensitivity_Test-covnet_lambda_Mu_preds_Lower_16.npy", preds_Lower_16)
    
np.save("Sensitivity_Test-covnet_lambda_Mu_mu_s.npy", mu_s)
np.save("Sensitivity_Test-covnet_lambda_Mu_preds_MEAN.npy", preds_MEAN)

#np.save("Sensitivity_Test-preds_VaryingMu_SINGLE_LAMBDA_MEAN.npy", preds_SINGLE_LAMBDA_MEAN)
#np.save("Sensitivity_Test-preds_VaryingMu_preds_SINGLE_LAMBDA_Lower_16", preds_SINGLE_LAMBDA_Lower_16)
#np.save("Sensitivity_Test-preds_VaryingMu_preds_SINGLE_LAMBDA_Upper_84", preds_SINGLE_LAMBDA_Upper_84)

In [ ]:
preds_Upper_84 = np.load("Sensitivity_Test-covnet_lambda_Mu_preds_Upper_84.npy")
preds_Lower_16 = np.load("Sensitivity_Test-covnet_lambda_Mu_preds_Lower_16.npy")
    
mu_s = np.load("Sensitivity_Test-covnet_lambda_Mu_mu_s.npy")
preds_MEAN = np.load("Sensitivity_Test-covnet_lambda_Mu_preds_MEAN.npy")

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigma_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigma_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

In [ ]:
fig = pylab.figure()
pylab.plot(mu_s, sigmas, color='r', label="predicted value");
pylab.axhline(y=0.1, color='black', linestyle='-', label="actual value")
pylab.fill_between(mu_s, sigma_lower_16, sigma_upper_84, color='r', alpha=0.1, edgecolor='none')
pylab.xlabel("Mu")
pylab.ylabel("Sigma")

pylab.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
pylab.show();
fig.savefig(ROOT_DIR + "/Images/Output-Sensitivity-Results/ConvolutionalNN-MultipleOutput-ELU/Varying-Mu/"+"sigma.png")

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

In [ ]:
plt.plot(mu_s, jump_sigmas, color='b', label="predicted value");
plt.axhline(y=0.07, color='black', linestyle='-', label="actual value")
plt.fill_between(mu_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
plt.xlabel("Mu")
plt.ylabel("Jumps Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

In [ ]:
plt.plot(mu_s, jump_mus, color='g', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(mu_s, jump_mus_lower_16, jump_mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Mu")
plt.ylabel("Jumps Mu")

plt.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
lambdas = np.array(preds_MEAN)[:,0].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

In [ ]:
lambdas_upper_84

In [ ]:
plt.plot(mu_s, lambdas, color='y', label="predicted value");
plt.axhline(y=0.02, color='black', linestyle='-', label="actual value")
plt.fill_between(mu_s, lambdas_lower_16, lambdas_upper_84, color='b', alpha=1, edgecolor='none')
plt.xlabel("Mu")
plt.ylabel("Lambda")

plt.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)

In [ ]:
plt.plot(mu_s, mus_, color='g', label="predicted value");
plt.plot(mu_s, mu_s, color='black', label="actual value");
plt.fill_between(mu_s, mus_lower_16, mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Mu")
plt.ylabel("Mu")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

# Sensitivity Test - Sigma

In [ ]:
sigma_s = []
preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []

for j in range(100):
    covnet_lambda = covnet_multiple_output_with_ELUs()
    covnet_lambda.load_weights(H5_DIR_COVNET_MULTIPLE_ELU)
    
    mp.all_sigma = 0.001 + j*(0.2 - 0.001)/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set()
    
    print("PREDICTION...\n")
    predictions_elu = covnet_lambda.predict(test_set_pred)
    
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    print("   sigma,    mu,      jump_sigma,      jump_mu,    lambda")
    print("   "+str(mp.all_sigma)+"      0.05       0.07              0.05       0.02")
    print(np.mean(predictions_elu, axis = 1)+np.std(predictions_elu, axis = 1))
    print(np.percentile(predictions_elu, 84, axis = 1))
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    sigma_s.append(mp.all_sigma)
    
np.save("Sensitivity_Test-covnet_lambda_Sigma_sigma_s.npy", sigma_s)
np.save("Sensitivity_Test-covnet_lambda_Sigma_preds_MEAN.npy", preds_MEAN)
np.save("Sensitivity_Test-covnet_lambda_Sigma_preds_Upper_84.npy", preds_Upper_84)
np.save("Sensitivity_Test-covnet_lambda_Sigma_preds_Lower_16.npy", preds_Lower_16)

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigma_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigma_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

plt.plot(sigma_s, sigmas, color='r', label="predicted value");
plt.plot(sigma_s, sigma_s, color='black', label="actual value");
plt.fill_between(sigma_s, sigma_lower_16, sigma_upper_84, color='r', alpha=0.1, edgecolor='none')
plt.xlabel("Sigma")
plt.ylabel("Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=0, borderaxespad=0.)
plt.show();

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

plt.plot(sigma_s, jump_sigmas, color='b', label="predicted value");
plt.axhline(y=0.07, color='black', linestyle='-', label="actual value")
plt.fill_between(sigma_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
plt.xlabel("Sigma")
plt.ylabel("Jumps Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

plt.plot(sigma_s, jump_mus, color='g', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(sigma_s, jump_mus_lower_16, jump_mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Sigma")
plt.ylabel("Jumps Mu")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
lambdas = np.array(preds_MEAN)[:,4].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,4].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,4].reshape(100)

plt.plot(sigma_s, lambdas, color='y', label="predicted value");
plt.axhline(y=0.02, color='black', linestyle='-', label="actual value")
plt.fill_between(sigma_s, lambdas_lower_16, lambdas_upper_84, color='y', alpha=0.1, edgecolor='none')
plt.xlabel("Sigma")
plt.ylabel("Lambda")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)

plt.plot(sigma_s, mus_, color='g', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(sigma_s, mus_lower_16, mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Sigma")
plt.ylabel("Mu")

plt.legend(bbox_to_anchor=(0.4085, 1), loc=1, borderaxespad=0.)
plt.show();

# Sensitivity Test - Jumps Sigma

In [ ]:
jump_sigma_s = []
preds_MEAN = []
preds_Lower_16 = []
preds_Upper_84 = []

for j in range(100):
    covnet_lambda = covnet_multiple_output_with_ELUs()
    covnet_lambda.load_weights(H5_DIR_COVNET_MULTIPLE_ELU)
    
    mp.jumps_sigma = 0.001 + j*(0.2 - 0.001)/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set()
    
    
    print("PREDICTION...\n")
    predictions_elu = covnet_lambda.predict(test_set_pred)
    
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    print("   sigma,    mu,      jump_sigma,      jump_mu,    lambda")
    print("   0.1      0.05       "+str(mp.jumps_sigma)+"              0.05       0.02")
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    jump_sigma_s.append(mp.jumps_sigma)
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
np.save("Sensitivity_Test-covnet_lambda_JumpSigma_preds_Upper_84.npy", preds_Upper_84)
np.save("Sensitivity_Test-covnet_lambda_JumpSigma_preds_Lower_16.npy", preds_Lower_16)
np.save("Sensitivity_Test-covnet_lambda_JumpSigma_jump_sigma_s.npy", jump_sigma_s)
np.save("Sensitivity_Test-covnet_lambda_JumpSigma_preds_MEAN.npy", preds_MEAN)

In [ ]:
preds_Upper_84 = np.load("Sensitivity_Test-covnet_lambda_JumpSigma_preds_Upper_84.npy")
preds_Lower_16 = np.load("Sensitivity_Test-covnet_lambda_JumpSigma_preds_Lower_16.npy")
jump_sigma_s = np.load("Sensitivity_Test-covnet_lambda_JumpSigma_jump_sigma_s.npy")
preds_MEAN = np.load("Sensitivity_Test-covnet_lambda_JumpSigma_preds_MEAN.npy")

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)

plt.plot(jump_sigma_s, mus_, color='g', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_sigma_s, mus_lower_16, mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Jumps Sigma")
plt.ylabel("Mu")

plt.legend(bbox_to_anchor=(1, 0.19), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigmas_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigmas_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

plt.plot(jump_sigma_s, sigmas, color='r', label="predicted value");
plt.axhline(y=0.1, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_sigma_s, sigmas_lower_16, sigmas_upper_84, color='r', alpha=0.1, edgecolor='none')
plt.xlabel("Jumps Sigma")
plt.ylabel("Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=0, borderaxespad=0.)
plt.show();

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

plt.plot(jump_sigma_s, jump_sigmas, color='b', label="predicted value");
plt.plot(jump_sigma_s, jump_sigma_s, color='black', label="actual value");
plt.fill_between(jump_sigma_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
plt.xlabel("Jumps Sigma")
plt.ylabel("Jumps Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

plt.plot(jump_sigma_s, jump_mus, color='g', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_sigma_s, jump_mus_lower_16, jump_mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Jumps Sigma")
plt.ylabel("Jumps Mu")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
lambdas = np.array(preds_MEAN)[:,4].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,4].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,4].reshape(100)

plt.plot(jump_sigma_s, lambdas, color='y', label="predicted value");
plt.axhline(y=0.02, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_sigma_s, lambdas_lower_16, lambdas_upper_84, color='y', alpha=0.2, edgecolor='none')
plt.xlabel("Jumps Sigma")
plt.ylabel("Lambda")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

# Sensitivity Test - Jumps Mu

In [ ]:
jump_mu_s = []
preds_MEAN = []
preds_Upper_84 = []
preds_Lower_16 = []

for j in range(100):
    covnet_lambda = covnet_multiple_output_with_ELUs()
    covnet_lambda.load_weights(H5_DIR_COVNET_MULTIPLE_ELU)
    
    mp.jumps_mu = -0.5 + j/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set()
    
    
    print("PREDICTION...\n")
    predictions_elu = covnet_lambda.predict(test_set_pred)
    
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    print("   sigma,    mu,      jump_sigma,      jump_mu,    lambda")
    print("   0.1      0.05       0.07              "+str(mp.jumps_mu)+"       0.02")
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    jump_mu_s.append(mp.jumps_mu)
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
np.save("Sensitivity_Test-covnet_lambda_JumpMu_preds_Upper_84.npy", preds_Upper_84)
np.save("Sensitivity_Test-covnet_lambda_JumpMu_preds_Lower_16.npy", preds_Lower_16)
    
np.save("Sensitivity_Test-covnet_lambda_JumpMu_jump_mu_s.npy", jump_mu_s)
np.save("Sensitivity_Test-covnet_lambda_JumpMu_preds_MEAN.npy", preds_MEAN)

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)

plt.plot(jump_mu_s, mus_, color='g', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_mu_s, mus_lower_16, mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Jumps Mu")
plt.ylabel("Mu")

plt.legend(bbox_to_anchor=(1, 0.19), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigmas_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigmas_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

plt.plot(jump_mu_s, sigmas, color='r', label="predicted value");
plt.axhline(y=0.1, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_mu_s, sigmas_lower_16, sigmas_upper_84, color='r', alpha=0.1, edgecolor='none')
plt.xlabel("Jumps Mu")
plt.ylabel("Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=0, borderaxespad=0.)
plt.show();

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

plt.plot(jump_mu_s, jump_sigmas, color='b', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_mu_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
plt.xlabel("Jumps Mu")
plt.ylabel("Jumps Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

plt.plot(jump_mu_s, jump_mus, color='g', label="predicted value");
plt.plot(jump_mu_s, jump_mu_s, color='black', label="actual value");
plt.fill_between(jump_mu_s, jump_mus_lower_16, jump_mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Jumps Mu")
plt.ylabel("Jumps Mu")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
lambdas = np.array(preds_MEAN)[:,4].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,4].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,4].reshape(100)

plt.plot(jump_mu_s, lambdas, color='y', label="predicted value");
plt.axhline(y=0.02, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_mu_s, lambdas_lower_16, lambdas_upper_84, color='y', alpha=0.2, edgecolor='none')
plt.xlabel("Jumps Mu")
plt.ylabel("Lambda")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

# Sensitivity Test - Lambda

In [ ]:
jump_lambda_s = []
preds_MEAN = []
preds_Upper_84 = []
preds_Lower_16 = []

for j in range(100):
    covnet_lambda = covnet_multiple_output_with_ELUs()
    covnet_lambda.load_weights(H5_DIR_COVNET_MULTIPLE_ELU)
    
    mp.lamda = 0.0001 + j*(0.025 - 0.0001)/100
    test_set = []
    
    print(j)
    
    print("GENERATION...")
    test_set_pred = standard_convolutional_test_set()
    
    
    print("PREDICTION...\n")
    predictions_elu = covnet_lambda.predict(test_set_pred)
    
    print(np.transpose(np.mean(predictions_elu, axis = 1)))
    print("   sigma,    mu,      jump_sigma,      jump_mu,    lambda")
    print("   0.1      0.05       0.07              0.05       "+str(mp.lamda))
    preds_MEAN.append(np.mean(predictions_elu, axis = 1))
    jump_lambda_s.append(mp.lamda)
    preds_Upper_84.append(np.percentile(predictions_elu, 84, axis = 1))
    preds_Lower_16.append(np.percentile(predictions_elu, 16, axis = 1))
    
np.save("Sensitivity_Test-covnet_lambda_Lambda_preds_Upper_84.npy", preds_Upper_84)
np.save("Sensitivity_Test-covnet_lambda_Lambda_preds_Lower_16.npy", preds_Lower_16)
    
np.save("Sensitivity_Test-covnet_lambda_Lambda_jump_lambda_s.npy", jump_lambda_s)
np.save("Sensitivity_Test-covnet_lambda_Lambda_preds_MEAN.npy", preds_MEAN)

In [ ]:
mus_ = np.array(preds_MEAN)[:,1].reshape(100)
mus_upper_84 = np.array(preds_Upper_84)[:,1].reshape(100)
mus_lower_16 = np.array(preds_Lower_16)[:,1].reshape(100)

plt.plot(jump_lambda_s, mus_, color='g', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_lambda_s, mus_lower_16, mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Lambda")
plt.ylabel("Mu")

plt.legend(bbox_to_anchor=(1, 0.19), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
sigmas = np.array(preds_MEAN)[:,0].reshape(100)
sigmas_upper_84 = np.array(preds_Upper_84)[:,0].reshape(100)
sigmas_lower_16 = np.array(preds_Lower_16)[:,0].reshape(100)

plt.plot(jump_lambda_s, sigmas, color='r', label="predicted value");
plt.axhline(y=0.1, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_lambda_s, sigmas_lower_16, sigmas_upper_84, color='r', alpha=0.1, edgecolor='none')
plt.xlabel("Lambda")
plt.ylabel("Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=0, borderaxespad=0.)
plt.show();

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)

plt.plot(jump_lambda_s, jump_sigmas, color='b', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_lambda_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
plt.xlabel("Lambda")
plt.ylabel("Jumps Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
jump_sigmas = np.array(preds_MEAN)[:,2].reshape(100)
jump_sigmas_upper_84 = np.array(preds_Upper_84)[:,2].reshape(100)
jump_sigmas_lower_16 = np.array(preds_Lower_16)[:,2].reshape(100)


plt.plot(jump_lambda_s, jump_sigmas, color='b', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_lambda_s, jump_sigmas_lower_16, jump_sigmas_upper_84, color='b', alpha=0.1, edgecolor='none')
plt.xlabel("Lambda")
plt.ylabel("Jumps Sigma")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
jump_mus = np.array(preds_MEAN)[:,3].reshape(100)
jump_mus_upper_84 = np.array(preds_Upper_84)[:,3].reshape(100)
jump_mus_lower_16 = np.array(preds_Lower_16)[:,3].reshape(100)

plt.plot(jump_lambda_s, jump_mus, color='g', label="predicted value");
plt.axhline(y=0.05, color='black', linestyle='-', label="actual value")
plt.fill_between(jump_lambda_s, jump_mus_lower_16, jump_mus_upper_84, color='g', alpha=0.1, edgecolor='none')
plt.xlabel("Lambda")
plt.ylabel("Jumps Mu")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

In [ ]:
lambdas = np.array(preds_MEAN)[:,4].reshape(100)
lambdas_upper_84 = np.array(preds_Upper_84)[:,4].reshape(100)
lambdas_lower_16 = np.array(preds_Lower_16)[:,4].reshape(100)

plt.plot(jump_lambda_s, lambdas, color='y', label="predicted value");
plt.plot(jump_lambda_s, jump_lambda_s, color='black', label="actual value");
plt.fill_between(jump_lambda_s, lambdas_lower_16, lambdas_upper_84, color='y', alpha=0.2, edgecolor='none')
plt.xlabel("Lambda")
plt.ylabel("Lambda")

plt.legend(bbox_to_anchor=(1, 0.185), loc=1, borderaxespad=0.)
plt.show();

# Tests

In [ ]:
test_set_pred, train_statistics = standard_test_sets()

In [ ]:
#predictions = covnet_mo.predict(test_set_pred)
predictions_elu = covnet_mo_elu.predict(test_set_pred)

In [ ]:
predictions_ff = feedforward.predict(train_statistics)

In [ ]:
np.mean(predictions_ff, axis = 1)

In [ ]:
np.mean(predictions_elu, axis = 1)

In [ ]:
mp = test_set_model_params()
print(mp.all_sigma, mp.gbm_mu, mp.jumps_sigma, mp.jumps_mu, mp.lamda)

In [ ]:
from PIL import Image

In [ ]:
def predictions_hist(predictions, true_value, plot_title = "model", param_name = "param", min_edge = None, max_edge = None, y_lim = (None, None), bin_size = 0.0025, img_subdir = "/Images/", save_to_file = True):
    
    if min_edge == None:
        min_edge = np.min(predictions)
    
    if max_edge == None:
        max_edge = np.max(predictions)
    
    bin_size = (max_edge-min_edge)/50
    N = (max_edge-min_edge)/bin_size; Nplus1 = N + 1
    bin_list = np.linspace(min_edge, max_edge, Nplus1)

    fig = pylab.figure();
    l = pylab.hist(predictions, bins=bin_list);
    #pylab.title(plot_title);
    pylab.axvline(x = true_value, c='k');
    pylab.xlim(min_edge, max_edge);
    
    if y_lim != (None, None):
        pylab.ylim(y_lim[0], y_lim[1])
    
    fig.set_dpi(120)     # draw the canvas, cache the renderer
    
    def fig2data ( fig ):
        """
        @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
        @param fig a matplotlib figure
        @return a numpy 3D array of RGBA values
        """
        # draw the renderer
        fig.canvas.draw ( )

        # Get the RGBA buffer from the figure
        w,h = fig.canvas.get_width_height()
        buf = numpy.fromstring ( fig.canvas.tostring_argb(), dtype=numpy.uint8 )
        buf.shape = ( w, h,4 )

        # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
        buf = numpy.roll ( buf, 3, axis = 2 )
        return buf

    def fig2img ( fig ):
        """
        @brief Convert a Matplotlib figure to a PIL Image in RGBA format and return it
        @param fig a matplotlib figure
        @return a Python Imaging Library ( PIL ) image
        """
        # put the figure pixmap into a numpy array
        buf = fig2data ( fig )
        w, h, d = buf.shape
        return Image.frombytes( "RGBA", ( w ,h ), buf.tostring( ) )
    
    #width, height = fig.get_size_inches() * fig.get_dpi()
    #img = (np.fromstring(fig.canvas.tostring_rgb(), dtype='uint8', sep='')).reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    if save_to_file:
        fig.savefig(ROOT_DIR + img_subdir + plot_title + "_" + param_name + ".png");
    return fig, l

In [ ]:
"""predictions_hist(predictions[0], mp.all_sigma, 
                 "Convolutional Architecture - Multiple Output - ReLU", 
                 "Sigma",
                 min_edge = 0.08,
                 max_edge = 0.155,
                 y_lim = (0, 210),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ReLU/");
"""
predictions_hist(predictions_elu[0], mp.all_sigma, 
                 "Convolutional Architecture - Multiple Output - ELU", 
                 "Sigma",
                 min_edge = 0.08,
                 max_edge = 0.155,
                 y_lim = (0, 210),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ELU/");

predictions_hist(predictions_ff[0], mp.all_sigma, 
                 "Fully Connected Architecture - Multiple Output - ELU", 
                 "Sigma",
                 min_edge = 0.08,
                 max_edge = 0.155,
                 img_subdir = "/Images/Parameter-Prediction-Results/FullyConnected-MultipleOutput-ELU/");

In [ ]:
"""predictions_hist(predictions[1], mp.gbm_mu, 
                 "Convolutional Architecture - Multiple Output - ReLU", 
                 "Mu",
                 min_edge = -0.1,
                 max_edge = 0.4,
                 y_lim = (0, 120),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ReLU/");
"""
predictions_hist(predictions_elu[1], mp.gbm_mu, 
                 "Convolutional Architecture - Multiple Output - ELU", 
                 "Mu",
                 min_edge = -0.1,
                 max_edge = 0.4,
                 y_lim = (0, 120),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ELU/");

predictions_hist(predictions_ff[1], mp.gbm_mu, 
                 "Fully Connected Architecture - Multiple Output - ELU", 
                 "Mu",
                 min_edge = -0.1,
                 max_edge = 0.4,
                 y_lim = (0, 120),
                 img_subdir = "/Images/Parameter-Prediction-Results/FullyConnected-MultipleOutput-ELU/");

In [ ]:
"""predictions_hist(predictions[2], mp.jumps_sigma, 
                 "Convolutional Architecture - Multiple Output - ReLU", 
                 "Jumps Sigma",
                 min_edge = 0.02,
                 max_edge = 0.13,
                 y_lim = (0, 80),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ReLU/");
"""
predictions_hist(predictions_elu[2], mp.jumps_sigma, 
                 "Convolutional Architecture - Multiple Output - ELU", 
                 "Jumps Sigma",
                 min_edge = 0.02,
                 max_edge = 0.13,
                 y_lim = (0, 80),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ELU/");

"""predictions_hist(predictions_ff[2], mp.jumps_sigma, 
                 "Fully Connected Architecture - Multiple Output - ELU", 
                 "Jumps Sigma",
                 min_edge = 0.02,
                 max_edge = 0.13,
                 img_subdir = "/Images/Parameter-Prediction-Results/FullyConnected-MultipleOutput-ELU/");"""

preds_so = covnet_jump_sigma.predict(test_set_pred)
predictions_hist(preds_so, mp.jumps_sigma, 
                 "Convolutional Architecture - Multiple Output - ELU", 
                 "Jumps Sigma",
                 min_edge = 0.02,
                 max_edge = 0.13,
                 y_lim = (0, 80),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ELU/");

In [ ]:
"""predictions_hist(predictions[3], mp.jumps_mu, 
                 "Convolutional Architecture - Multiple Output - ReLU", 
                 "Jumps Mu",
                 min_edge = -0.1,
                 max_edge = 0.15,
                 y_lim = (0, 120),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ReLU/");
"""
predictions_hist(predictions_elu[3], mp.jumps_mu, 
                 "Convolutional Architecture - Multiple Output - ELU", 
                 "Jumps Mu",
                 min_edge = -0.1,
                 max_edge = 0.15,
                 y_lim = (0, 120),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ELU/");

predictions_hist(predictions_so, mp.jumps_mu, 
                 "Convolutional Architecture - Single Output - ELU", 
                 "Jumps Mu",
                 min_edge = -0.1,
                 max_edge = 0.15,
                 y_lim = (0, 120),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-SingleOutput/");

predictions_hist(predictions_ff[3], mp.jumps_mu, 
                 "Fully Connected Architecture - Multiple Output - ELU", 
                 "Jumps Mu",
                 min_edge = -0.1,
                 max_edge = 0.15,
                 y_lim = (0, 120),
                 img_subdir = "/Images/Parameter-Prediction-Results/FullyConnected-MultipleOutput-ELU/");

In [ ]:
predictions_so = covnet_jump_mu.predict(test_set_pred)

In [ ]:
predictions_so

In [ ]:
predictions_hist(np.reshape(predictions_so, len(predictions_so)), mp.jumps_mu, 
                 "Fully Connected Architecture - Multiple Output - ELU", 
                 "Jumps Mu",
                 min_edge = -0.1,
                 max_edge = 0.15,
                 y_lim = (0, 120),
                 img_subdir = "/Images/Parameter-Prediction-Results/FullyConnected-MultipleOutput-ELU/");


In [ ]:
single_param_predictions_lambda = covnet_lambda.predict(test_set_pred)

In [ ]:
"""predictions_hist(predictions[4], mp.lamda, 
                 "Convolutional Architecture - Multiple Output - ReLU", 
                 "Lambda",
                 min_edge = 0.004,
                 max_edge = 0.025,
                 y_lim = (0, 100),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ReLU/");
"""

predictions_hist(predictions_elu[4], mp.lamda, 
                 "Convolutional Architecture - Multiple Output - ELU", 
                 "Lambda",
                 min_edge = 0.004,
                 max_edge = 0.025,
                 y_lim = (0, 100),
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-MultipleOutput-ELU/");

predictions_hist(predictions_ff[4], mp.lamda, 
                 "Fully Connected Architecture - Multiple Output - ELU", 
                 "Lambda",
                 min_edge = 0.004,
                 max_edge = 0.025,
                 img_subdir = "/Images/Parameter-Prediction-Results/FullyConnected-MultipleOutput-ELU/");

predictions_hist(single_param_predictions_lambda, mp.lamda, 
                 "Convolutional Architecture - Single Output - ELU", 
                 "Lambda",
                 min_edge = 0.004,
                 max_edge = 0.025,
                 img_subdir = "/Images/Parameter-Prediction-Results/ConvolutionalNN-SingleOutput/");

In [ ]:
predictionset = []
for return_set in test_set:
    prediction = []
    output1 = covnet_mo.predict(np.array([return_set]))
    output2 = covnet_lambda.predict(np.array([return_set]))
    output3 = covnet_jump_mu.predict(np.array([return_set]))
    output4 = covnet_jump_sigma.predict(np.array([return_set]))
    for output in output1:
        prediction.append(output[0][0])
    prediction.append(output2[0][0])
    prediction.append(output3[0][0])
    prediction.append(output4[0][0])
    predictionset.append(prediction)
        

In [ ]:
preds = ff_ensembling.predict(np.array(predictionset))
np.transpose(preds)[4]

In [ ]:
lambdas = np.transpose(preds)[4]

bin_size = 0.0005; min_edge = np.min(lambdas)-0.003; max_edge = np.max(lambdas)
N = (max_edge-min_edge)/bin_size; Nplus1 = N + 1
bin_list = np.linspace(min_edge, max_edge, Nplus1)

plt.hist(lambdas, bins=bin_list);
plt.title("Single Output:  Lambda with true value = "+str(mp.lamda));
plt.axvline(x=mp.lamda, c='k')
plt.show();

In [ ]:
jump_mus = np.transpose(preds)[3]

bin_size = 0.0025; min_edge = np.min(jump_mus)-0.03; max_edge = np.max(jump_mus)
N = (max_edge-min_edge)/bin_size; Nplus1 = N + 1
bin_list = np.linspace(min_edge, max_edge, Nplus1)

plt.hist(jump_mus, bins=bin_list);
plt.axvline(x=mp.jumps_mu, c='k')
plt.title("Multiple Output: Mu_jumps with true value = "+str(mp.jumps_mu));
plt.show();

In [ ]:
jump_sigmas = np.transpose(preds)[2]

bin_size = 0.0025; min_edge = np.min(jump_sigmas)-0.025; max_edge = np.max(jump_sigmas)
N = (max_edge-min_edge)/bin_size; Nplus1 = N + 1
bin_list = np.linspace(min_edge, max_edge, Nplus1)

plt.hist(jump_sigmas, bins=bin_list);
plt.title("Multiple Output: Sigma_jumps with true value = "+str(mp.jumps_sigma));
plt.axvline(x=mp.jumps_sigma, c='k')
plt.show();

In [ ]:
preds

In [ ]:
random_seed = System.currentTimeMillis()
print(random_seed)

In [ ]:
mp = random_model_params()
mp.gbm_mu = 0.1
mp.all_sigma = 0.1
mp.lamda = 0.02
mp.jumps_mu = 0.1
mp.jumps_sigma = 0.05

nrand.seed(random_seed)
random.seed(random_seed)
RETURNS_ = np.exp(geometric_brownian_motion_jump_diffusion_log_returns(mp))

In [ ]:
path1 = [1]
for i in range(len(RETURNS_)):
    path1.append(path1[i]*RETURNS_[i])

In [ ]:
nrand.seed(random_seed)
random.seed(random_seed)
return_set = np.reshape(np.array(geometric_brownian_motion_jump_diffusion_log_returns(mp)), (40, 50, 1))

output1 = covnet_mo.predict(np.array([return_set]))
output2 = covnet_lambda.predict(np.array([return_set]))
output3 = covnet_jump_mu.predict(np.array([return_set]))
output4 = covnet_jump_sigma.predict(np.array([return_set]))
prediction = []

for output in output1:
    prediction.append(output[0][0])
prediction.append(output2[0][0])
prediction.append(output3[0][0])
prediction.append(output4[0][0])    

In [ ]:
preds = covnet_mo_elu.predict(np.array([return_set]))

In [ ]:
mp.all_sigma = preds[0][0][0]
mp.gbm_mu = preds[1][0][0]
mp.jumps_sigma = preds[2][0][0]
mp.jumps_mu = preds[3][0][0]
mp.lamda = preds[4][0][0]

In [ ]:
preds

In [ ]:
nrand.seed(random_seed)
random.seed(random_seed)
RETURNS_ = np.exp(geometric_brownian_motion_jump_diffusion_log_returns(mp))
path2 = [1]
for i in range(len(RETURNS_)):
    path2.append(path2[i]*RETURNS_[i])
plt.plot(path2);
plt.plot(path1);